# Comparaison de la sélection de variables avec STABL (Random Forest, XGBoost, Lasso, ElasticNet)

Ce notebook compare la sélection de variables et les performances de classification/régression entre STABL utilisant des modèles d’arbres (Random Forest, XGBoost) avec grid search, et STABL utilisant des modèles linéaires (Lasso, ElasticNet). Les résultats sont évalués par validation croisée, en termes de performances et de nombre de variables sélectionnées.

In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression, Lasso, ElasticNet
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error
from stabl.stabl import Stabl
from stabl.adaptive import ALasso, ALogitLasso
from stabl.preprocessing import LowInfoFilter
# from stabl.visualization import plot_fdr_graph
from sklearn.feature_selection import VarianceThreshold
from sklearn.base import clone
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


## 1. Charger et explorer les données

Nous allons charger un jeu de données tabulaire (exemple : CyTOF.csv et outcome.csv), afficher ses dimensions, types de variables, et quelques statistiques descriptives.

In [ ]:
# Charger les données (exemple : CyTOF.csv et outcome.csv)
features = pd.read_csv("data/COVID-19/Training/Proteomics.csv")
outcome = pd.read_csv("data/COVID-19/Training/Mild&ModVsSevere.csv")

print("Features shape:", features.shape)
print("Outcome shape:", outcome.shape)
print("Features columns:", features.columns.tolist()[:10], "...")
print("Outcome columns:", outcome.columns.tolist())

# Statistiques descriptives
print(features.describe())
print(outcome.describe())

# Afficher la distribution de la variable cible (exemple : model1b)
outcome["Mild&ModVsSevere"].value_counts().plot(kind="bar")
plt.title("Distribution de la variable cible")
plt.show()

## 2. Définir le préprocessing

Pipeline de préprocessing : imputation, standardisation, filtrage faible variance, filtrage low info.

## Configuration des pipelines STABL avec modèles d’arbres et linéaires

Dans cette section, nous allons configurer plusieurs pipelines de sélection de variables et de modélisation :
- **STABL + Random Forest** : sélection de variables avec STABL utilisant Random Forest comme estimateur, avec recherche de grille (grid search) pour optimiser les hyperparamètres, et validation croisée (cross-validation).
- **STABL + XGBoost** : même principe, mais avec XGBoost comme estimateur.
- **STABL + Lasso** : pipeline linéaire utilisant Lasso.
- **STABL + ElasticNet** : pipeline linéaire utilisant ElasticNet.

Pour chaque pipeline, nous appliquerons la sélection de variables, puis nous évaluerons les performances en cross-validation et le nombre de variables sélectionnées. Les résultats seront comparés à la fin du notebook.

## Cross-validation, sauvegarde des résultats et visualisation

Pour chaque modèle, nous allons :
- Effectuer une cross-validation (stratifiée) sur le dataset.
- Sauvegarder les courbes ROC et PR dans le dossier `Benchmarks results/` avec le nom du modèle et du dataset.
- Sauvegarder les importances des variables sélectionnées et leur nombre.
- Comparer les performances (AUC, accuracy, etc.) et le nombre de variables sélectionnées entre tous les modèles.

Les résultats seront visualisés sous forme de tableaux et de graphes pour faciliter la comparaison.

## Conclusion

Ce notebook permet de comparer la sélection de variables et les performances de classification entre STABL (Random Forest, XGBoost, Lasso, ElasticNet) sur le dataset CyTOF. Les courbes ROC, PR, importances des variables et tableaux de résultats sont sauvegardés dans le dossier `Benchmarks results/` avec des noms explicites pour chaque modèle et dataset. Vous pouvez adapter ce pipeline à d'autres datasets en modifiant la section de chargement des données.

In [ ]:
# Pipeline de préprocessing (à utiliser dans chaque pipeline)
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from stabl.preprocessing import LowInfoFilter

preprocessing = Pipeline([
    ("variance_threshold", VarianceThreshold(threshold=0)),
    ("low_info_filter", LowInfoFilter(max_nan_fraction=0.2)),
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

In [ ]:
def load_maigane(data_path):
    """
    Charge le dataset Maigane (sans split) et renvoie:
      (train_data_dict, None, y_train, None, None, 'binary')

    Hypothèses:
      - Outcome: ControlVsEM.csv avec colonnes 'sampleID,EM' (Control/EM)
      - Omics: 1ère colonne = ID (souvent sans nom / 'Unnamed: 0' / "" / quotes),
               valeurs style 'slide1_P01endoIMC_mgd_...'
      - Exclusions gérées en amont (ici on charge uniquement 4 fichiers utiles)
      - Préprocessing: *sanitisation systématique* des noms de features (remplace / \ : * ? " < > | par _),
                       supprime les quotes éventuelles, déduplique les doublons de noms.
    """

    def _sanitize_cols(cols):
        def clean(c):
            s = str(c)
            s = s.strip().strip('"').strip("'")
            s = re.sub(r'[\\/:*?"<>|]+', '_', s)          # interdit → _
            s = re.sub(r'\s+', ' ', s).strip()            # espaces multiples
            s = s.replace(' ', '_')
            s = re.sub(r'_+', '_', s)                     # underscores multiples
            if s == '' or s.lower().startswith('unnamed'):
                s = 'feature'
            return s
        cleaned = [clean(c) for c in cols]
        # déduplication
        seen = {}
        uniq = []
        for s in cleaned:
            if s not in seen:
                seen[s] = 1
                uniq.append(s)
            else:
                uniq.append(f"{s}__{seen[s]}")
                seen[s] += 1
        return uniq

    def _read_omic(path):
        # lecture tolérante, gère colonnes sans nom / "" / "Unnamed: 0"
        df0 = pd.read_csv(path, engine="python", sep=None, na_values=["NA", "NaN", ""], keep_default_na=True)
        first = df0.columns[0]
        cond_unnamed = (first in ("", " ", "Unnamed: 0", "Unnamed: 0.1", "Unnamed: 1"))
        cond_looks_like_slide = df0[first].astype(str).str.startswith("slide").any()
        if cond_unnamed or cond_looks_like_slide:
            df = df0.set_index(first)
        else:
            df = pd.read_csv(path, engine="python", sep=None, na_values=["NA", "NaN", ""], keep_default_na=True, index_col=0)

        # nettoie index (IDs)
        df.index = df.index.astype(str).str.strip().str.strip('"').str.strip("'")
        df = df[~df.index.duplicated(keep="first")]

        # nettoie/sanitise toutes les colonnes (qu'elles soient entre guillemets ou non)
        df.columns = _sanitize_cols(df.columns)

        # ne garder que les colonnes numériques pour le modèle
        return df.select_dtypes(include=[np.number])

    # 1) Outcome
    ydf = pd.read_csv(join(data_path, "ControlVsEM.csv"), na_values=["NA", "NaN", ""], keep_default_na=True)
    if "sampleID" not in ydf.columns or "EM" not in ydf.columns:
        raise ValueError("ControlVsEM.csv doit contenir les colonnes 'sampleID' et 'EM'.")
    y = ydf.set_index("sampleID")["EM"]
    if y.dtype.kind not in "iu":
        y = y.map({"Control": 0, "EM": 1}).astype(int)
    y.index = y.index.astype(str).str.strip().str.strip('"').str.strip("'")

    # 2) Omics
    file_map = {
        "CellDensities":  "EMIMCmdiop_celldensities.csv",
        "Functional":     "EMIMCmdiop_functional.csv",
        "Metavariables":  "EMIMCmdiop_metavariables.csv",
        "Neighborhood":   "EMIMCmdiop_neighborhood.csv",
    }
    X = {k: _read_omic(join(data_path, fname)) for k, fname in file_map.items()}

    # 3) Aligner sur IDs communs
    common = y.index.copy()
    for df in X.values():
        common = common.intersection(df.index)
    if common.empty:
        raise ValueError("Aucun échantillon commun entre omiques et labels.")
    y = y.loc[common]
    X = {k: df.loc[common] for k, df in X.items()}

    groups=pd.Series([e.split("_")[3] for e in y.index], index=y.index)

    return X, None, y, None, groups, "binary"

X_train, _, y_train, _, groups, task_type = load_maigane("./data/Maigane data")

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# ── 1) Charger les données
X_train, _, y_train, _, groups, task_type = load_maigane("./data/Maigane data")

# ── 2) Résumé par omique
summary_rows = []
for k, df in X_train.items():
    n_samples, n_feats = df.shape
    miss_rate = df.isna().mean().mean() if n_feats else np.nan
    zeros_rate = (df == 0).sum().sum() / df.size if df.size else np.nan
    summary_rows.append({
        "omic": k,
        "n_samples": n_samples,
        "n_features": n_feats,
        "mean_missing_rate": round(float(miss_rate), 4),
        "overall_zeros_rate": round(float(zeros_rate), 4),
    })
summary_df = pd.DataFrame(summary_rows).set_index("omic")
print("\nRésumé omiques:\n", summary_df)

# ── 3) Plots (matplotlib pur)
out_dir = "viz_maigane"
os.makedirs(out_dir, exist_ok=True)

# 3a. Répartition des classes
plt.figure(figsize=(5,4))
counts = y_train.value_counts().sort_index()
labels = ["Control" if i==0 else "EM" for i in counts.index]
plt.bar(labels, counts.values)
plt.title("Répartition des classes (y)")
plt.xlabel("Classe"); plt.ylabel("Effectif")
plt.tight_layout(); plt.savefig(os.path.join(out_dir, "classes.png")); plt.close()

# 3b. Répartition des groupes (top 20)
grp_counts = groups.value_counts()
top_n = 20 if len(grp_counts) > 20 else len(grp_counts)
plt.figure(figsize=(10,4))
plt.bar(grp_counts.index[:top_n].astype(str), grp_counts.values[:top_n])
plt.title(f"Groupes (top {top_n})")
plt.xticks(rotation=90); plt.ylabel("Effectif")
plt.tight_layout(); plt.savefig(os.path.join(out_dir, "groupes_top20.png")); plt.close()

# 3c. Distribution des valeurs (clippée aux 1er/99e pct) par omique
for k, df in X_train.items():
    vals = df.values.ravel()
    vals = vals[~np.isnan(vals)]
    if vals.size == 0:
        continue
    lo, hi = np.percentile(vals, [1, 99])
    vals = np.clip(vals, lo, hi)
    plt.figure(figsize=(6,4))
    plt.hist(vals, bins=60)
    plt.title(f"Distribution des valeurs — {k}")
    plt.xlabel("Valeurs (clippées 1–99e pct)"); plt.ylabel("Fréquence")
    plt.tight_layout(); plt.savefig(os.path.join(out_dir, f"hist_{k}.png")); plt.close()

# 3d. Histogramme des taux de NA par feature
for k, df in X_train.items():
    if df.shape[1] == 0:
        continue
    miss = df.isna().mean(axis=0).values
    plt.figure(figsize=(6,4))
    plt.hist(miss, bins=30)
    plt.title(f"Taux de valeurs manquantes par feature — {k}")
    plt.xlabel("Fraction de NA"); plt.ylabel("#Features")
    plt.tight_layout(); plt.savefig(os.path.join(out_dir, f"missing_{k}.png")); plt.close()

# ── 4) Préprocessing léger (imputation médiane + standardisation) et concaténation
X_prep = {}
imputers = {}
scalers = {}

for k, df in X_train.items():
    imp = SimpleImputer(strategy="median")
    X_imp = imp.fit_transform(df)
    imputers[k] = imp

    sc = StandardScaler()
    X_std = sc.fit_transform(X_imp)
    scalers[k] = sc

    X_prep[k] = pd.DataFrame(X_std, index=df.index, columns=[f"{k}__{c}" for c in df.columns])

X_all = pd.concat(X_prep.values(), axis=1)

# ── 5) Sauvegardes
os.makedirs("processed_maigane", exist_ok=True)
summary_df.to_csv("processed_maigane/omics_summary.csv")
X_all.to_parquet("processed_maigane/X_all_preprocessed.parquet")
y_train.to_csv("processed_maigane/y_train.csv")
groups.to_csv("processed_maigane/groups.csv")
print("\nFichiers écrits dans ./processed_maigane et figures dans ./viz_maigane")


In [ ]:
# ===============================
# 1. Imports et configuration générale
# ===============================
import os, re
import shutil
import numpy as np
import pandas as pd
from stabl import data
from stabl.multi_omic_pipelines import multi_omic_stabl_cv, multi_omic_stabl_cv_noe
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from stabl.stabl import Stabl
from stabl.adaptive import ALogitLasso, ALasso
from sklearn.linear_model import Lasso
from sklearn.base import clone
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupShuffleSplit

np.random.seed(42)

# ===============================
# 2. Définition des splits de validation croisée
# ===============================
# Outer CV pour l'évaluation globale, inner CV pour la recherche d'hyperparamètres
outer_cv = GroupShuffleSplit(n_splits=25, test_size=0.2, random_state=42)
#outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
inner_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

# ===============================
# 3. Définition des estimateurs et grilles d'hyperparamètres
# ===============================
artificial_type = "knockoff"  # ou "random_permutation" ou "knockoff"

# # Classification

# # Lasso
# lasso = LogisticRegression(penalty="l1", class_weight="balanced", max_iter=int(1e6), solver="liblinear", random_state=42)
# lasso_cv = GridSearchCV(lasso, param_grid={"C": np.logspace(-2, 2, 30)}, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

# # ALasso
# alasso = ALogitLasso(n_iter_lasso=2, penalty="l1", solver="liblinear", class_weight="balanced", max_iter=int(1e6), random_state=42)
# alasso_cv = GridSearchCV(alasso, param_grid={"C": np.logspace(-3, 2, 20)}, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

# ElasticNet
en = LogisticRegression(penalty='elasticnet', solver='saga', class_weight='balanced', max_iter=int(1e4), random_state=42)
en_params = {"C": np.logspace(-2, 1, 10), "l1_ratio": [0.5, 0.7, 0.9]}
en_cv = GridSearchCV(en, param_grid=en_params, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

# # RandomForest
# rf = RandomForestClassifier(random_state=42, max_features=0.2)
# rf_grid = {"max_depth": [3, 5, 7, 9, 11]}
# rf_cv = GridSearchCV(rf, scoring='roc_auc', param_grid=rf_grid, cv=inner_cv, n_jobs=-1)

# # XGBoost
# xgb = XGBClassifier(n_estimators=200, learning_rate=0.01, subsample=0.8, colsample_bytree=0.8, colsample_bynode=0.9, importance_type="shap", n_jobs=-1)
# xgb_grid = {"max_depth": [3, 6, 9], "reg_alpha": [0, 0.1, 0.5, 1]}
# xgb_cv = GridSearchCV(xgb, scoring='roc_auc', param_grid=xgb_grid, cv=inner_cv, n_jobs=-1)

#CatBoost
# cb = CatBoostClassifier(random_state=42, iterations=500, verbose=False)
# cb_grid = {"depth": [3, 5, 7], "learning_rate": [0.01, 0.1, 0.2], "l2_leaf_reg": [1, 3, 5]}
# cb_cv = GridSearchCV(cb, scoring='roc_auc', param_grid=cb_grid, cv=inner_cv, n_jobs=-1)

# Regression 
lasso = Lasso(max_iter=int(1e6), random_state=42)
lasso_cv = GridSearchCV(lasso, param_grid={"alpha": np.logspace(-2, 2, 30)}, scoring="r2", cv=inner_cv, n_jobs=-1)

# ALasso
alasso = ALasso(max_iter=int(1e6), random_state=42)
alasso_cv = GridSearchCV(alasso, scoring="r2", param_grid={"alpha": np.logspace(-2, 2, 30)}, cv=inner_cv, n_jobs=-1)

xgb = XGBRegressor(n_estimators=200, importance_type="gain", n_jobs=-1)
xgb_grid = {"max_depth": [3, 6, 9], "reg_alpha": [0, 0.1, 0.5, 1]}
xgb_cv = GridSearchCV(xgb, scoring='roc_auc', param_grid=xgb_grid, cv=inner_cv, n_jobs=-1)


# ===============================
# 4. Définition des estimateurs STABL
# ===============================

# # STABL Binaires

# stabl_lasso = Stabl(
#     base_estimator=lasso,
#     n_bootstraps=1000,
#     artificial_type=artificial_type,
#     artificial_proportion=1,
#     replace=False,
#     fdr_threshold_range=np.arange(0.1, 1, 0.01),
#     sample_fraction=0.5,
#     random_state=42,
#     lambda_grid={"C": np.linspace(0.01, 1, 10)},
#     verbose=1
# )

# stabl_alasso = Stabl(
#     base_estimator=alasso,
#     n_bootstraps=1000,
#     artificial_type=artificial_type,
#     artificial_proportion=1.0,
#     replace=False,
#     fdr_threshold_range=np.arange(0.01, 1, 0.01),
#     sample_fraction=0.5,
#     random_state=42,
#     lambda_grid={"C": np.logspace(-2, 2, 10)},
#     verbose=1,
# )

# stabl_en = clone(stabl_lasso).set_params(
#     base_estimator=en,
#     n_bootstraps=500,
#     lambda_grid=[{"C": np.logspace(-2, 1, 5), "l1_ratio": [0.5, 0.9]}],
#     verbose=1
# )

# stabl_rf = clone(stabl_lasso).set_params(
#     base_estimator=rf,
#     n_bootstraps=200,
#     lambda_grid=rf_grid,
#     verbose=1
# )

# stabl_xgb = clone(stabl_lasso).set_params(
#     base_estimator=xgb,
#     #bootstrap_threshold="median",
#     n_bootstraps=100,
#     fdr_threshold_range=np.arange(0.01, 1, 0.01),
#     lambda_grid=[xgb_grid],
#     verbose=1
# )

# STABL Régression

stabl_lasso = Stabl(
    base_estimator=lasso,
    n_bootstraps=500,
    artificial_type=artificial_type,
    artificial_proportion=1.,
    replace=False,
    fdr_threshold_range=np.arange(0.01, 1, 0.01),
    sample_fraction=0.5,
    random_state=42,
    lambda_grid={"alpha": np.logspace(0, 2, 10)},
    verbose=1
)

stabl_alasso = clone(stabl_lasso).set_params(
    base_estimator=alasso,
    lambda_grid={"alpha": np.logspace(0, 2, 10)},
    verbose=1
)

stabl_xgb = clone(stabl_lasso).set_params(
    base_estimator=xgb,
    n_bootstraps=100,
    lambda_grid=[xgb_grid],
    verbose=1
)

stabl_en = clone(stabl_lasso).set_params(
    base_estimator=en,
    n_bootstraps=500,
    lambda_grid=[{"C": np.logspace(-2, 1, 5), "l1_ratio": [0.5, 0.9]}],
    verbose=1
)

# stabl_cb = clone(stabl_lasso).set_params(
#     base_estimator=cb,
#     n_bootstraps=100,
#     lambda_grid=[cb_grid],
#     verbose=1
# )

# ===============================
# 5. Dictionnaire des estimateurs pour le benchmark
# ===============================
estimators = {
    "lasso": lasso_cv,
    "alasso": alasso_cv,
    "en": en_cv,
    "xgb": xgb_cv,
#    "rf": rf_cv,
    "stabl_lasso": stabl_lasso,
    "stabl_alasso": stabl_alasso,
    "stabl_en": stabl_en,
    "stabl_xgb": stabl_xgb,
#    "stabl_rf": stabl_rf,
}

models = [
    "STABL Lasso",
    "STABL ALasso",
    "STABL XGBoost",
#    "STABL ElasticNet",
#    "STABL RandomForest",
]


# ===============================
# 6. Chargement des données
# ===============================
#X_train, X_valid, y_train, y_valid, ids, task_type = data.load_covid_19("/Users/noeamar/Documents/Stanford/data/COVID-19")
#X_train, X_valid, y_train, y_valid, ids, task_type = data.load_ssi("data/Biobank SSI")
#X_train, X_valid, y_train, y_valid, ids, task_type = data.load_cfrna("data/CFRNA")
#X_train, X_valid, y_train, y_valid, ids, task_type = data.load_dream("data/Dream")
X_train, X_valid, y_train, y_valid, ids, task_type = data.load_onset_of_labor_EGA("data/Onset of Labor")

# # Covid-19
# # Assainir les labels
# # Harmoniser X_train / X_valid pour avoir mêmes lignes + colonnes
# if isinstance(X_train, dict):
#     common_keys = sorted(set(X_train) & set(X_valid))
#     X_train = {k: X_train[k].reindex(index=y_train.index, columns=X_train[k].columns.intersection(X_valid[k].columns)) for k in common_keys}
#     X_valid = {k: X_valid[k].reindex(index=y_valid.index, columns=X_train[k].columns) for k in common_keys}
# else:
#     common_cols = X_train.columns.intersection(X_valid.columns)
#     X_train = X_train.reindex(index=y_train.index, columns=common_cols)
#     X_valid = X_valid.reindex(index=y_valid.index, columns=common_cols)

# y = y_train.astype(int)
# vals = sorted(set(y.unique()))
# print("Labels uniques:", vals)
# assert set(vals) <= {0, 1}

# print("Shapes avant préprocessing :")
# print({k: df.shape for k, df in X_train.items()})  # aperçu des shapes

# # Séparer CyTOF et Proteomics
# cyto = X_train["CyTOF"]
# prot = X_train["Proteomics"].copy()

# # Application du préprocessing sur chaque omique, en gardant tous les patients
# # y inchangé (même index que data.load_ssi)
# y_train = y
# common = y_train.index.copy()
# for df in (cyto, prot):
#     common = common.intersection(df.index)
# X_train = {"CyTOF" : cyto.loc[common], "Proteomics" : prot.loc[common]}

# print("Shapes après préprocessing :")
# print({k: df.shape for k, df in X_train.items()})
# print("Labels :", y_train.shape, y_train.unique())

# # 2. Isoler le gros DataFrame CyTOF + DataFrame Proteomics
# cytof_full = X_train.pop("CyTOF")          # retire temporairement
# prot_df    = X_train.pop("Proteomics")     # retire temporairement

# # 3. Décomposer CyTOF en 3 sous-ensembles
# stim_map = {"unstim": "Unstim", "lps": "LPS", "il246": "IL246", "tnf": "TNF"}
# X_train  = {}                              # on reconstruit le dict final

# for short, nice in stim_map.items():
#     regex = re.compile(fr"(^|_)({short})(_|$)", flags=re.IGNORECASE)
#     cols  = [c for c in cytof_full.columns if regex.search(c)]
#     if not cols:
#         raise ValueError(f"Aucune colonne pour la stimulation « {short} »")
#     X_train[nice] = cytof_full[cols].copy()

# # 4. Ajouter Proteomics comme 4ᵉ entrée
# X_train["Proteomics"] = prot_df.copy()

# 5. Garder UNIQUEMENT les échantillons communs aux 4 DataFrames + y
# common = y_train.index.copy()
# for df in X_train.values():
#     common = common.intersection(df.index)
# if common.empty:
#     raise ValueError("Aucun échantillon commun entre omiques et labels.")

# for k in X_train:
#     X_train[k] = X_train[k].loc[common].copy()
# y_train = y_train.loc[common]

# print({k: df.shape for k, df in X_train.items()})  # aperçu des shapes

# # Maigane dataset

# from os.path import join

# def load_maigane(data_path):
#     """
#     Charge le dataset Maigane (sans split) et renvoie:
#       (train_data_dict, None, y_train, None, None, 'binary')

#     Hypothèses:
#       - Outcome: ControlVsEM.csv avec colonnes 'sampleID,EM' (Control/EM)
#       - Omics: 1ère colonne = ID (souvent sans nom / 'Unnamed: 0' / "" / quotes),
#                valeurs style 'slide1_P01endoIMC_mgd_...'
#       - Exclusions gérées en amont (ici on charge uniquement 4 fichiers utiles)
#       - Préprocessing: *sanitisation systématique* des noms de features (remplace / \ : * ? " < > | par _),
#                        supprime les quotes éventuelles, déduplique les doublons de noms.
#     """

#     def _sanitize_cols(cols):
#         def clean(c):
#             s = str(c)
#             s = s.strip().strip('"').strip("'")
#             s = re.sub(r'[\\/:*?"<>|]+', '_', s)          # interdit → _
#             s = re.sub(r'\s+', ' ', s).strip()            # espaces multiples
#             s = s.replace(' ', '_')
#             s = re.sub(r'_+', '_', s)                     # underscores multiples
#             if s == '' or s.lower().startswith('unnamed'):
#                 s = 'feature'
#             return s
#         cleaned = [clean(c) for c in cols]
#         # déduplication
#         seen = {}
#         uniq = []
#         for s in cleaned:
#             if s not in seen:
#                 seen[s] = 1
#                 uniq.append(s)
#             else:
#                 uniq.append(f"{s}__{seen[s]}")
#                 seen[s] += 1
#         return uniq

#     def _read_omic(path):
#         # lecture tolérante, gère colonnes sans nom / "" / "Unnamed: 0"
#         df0 = pd.read_csv(path, engine="python", sep=None, na_values=["NA", "NaN", ""], keep_default_na=True)
#         first = df0.columns[0]
#         cond_unnamed = (first in ("", " ", "Unnamed: 0", "Unnamed: 0.1", "Unnamed: 1"))
#         cond_looks_like_slide = df0[first].astype(str).str.startswith("slide").any()
#         if cond_unnamed or cond_looks_like_slide:
#             df = df0.set_index(first)
#         else:
#             df = pd.read_csv(path, engine="python", sep=None, na_values=["NA", "NaN", ""], keep_default_na=True, index_col=0)

#         # nettoie index (IDs)
#         df.index = df.index.astype(str).str.strip().str.strip('"').str.strip("'")
#         df = df[~df.index.duplicated(keep="first")]

#         # nettoie/sanitise toutes les colonnes (qu'elles soient entre guillemets ou non)
#         df.columns = _sanitize_cols(df.columns)

#         # ne garder que les colonnes numériques pour le modèle
#         return df.select_dtypes(include=[np.number])

#     # 1) Outcome
#     ydf = pd.read_csv(join(data_path, "ControlVsEM.csv"), na_values=["NA", "NaN", ""], keep_default_na=True)
#     if "sampleID" not in ydf.columns or "EM" not in ydf.columns:
#         raise ValueError("ControlVsEM.csv doit contenir les colonnes 'sampleID' et 'EM'.")
#     y = ydf.set_index("sampleID")["EM"]
#     if y.dtype.kind not in "iu":
#         y = y.map({"Control": 0, "EM": 1}).astype(int)
#     y.index = y.index.astype(str).str.strip().str.strip('"').str.strip("'")

#     # 2) Omics
#     file_map = {
#         "CellDensities":  "EMIMCmdiop_celldensities.csv",
#         "Functional":     "EMIMCmdiop_functional.csv",
#         "Metavariables":  "EMIMCmdiop_metavariables.csv",
#         "Neighborhood":   "EMIMCmdiop_neighborhood.csv",
#     }
#     X = {k: _read_omic(join(data_path, fname)) for k, fname in file_map.items()}

#     # 3) Aligner sur IDs communs
#     common = y.index.copy()
#     for df in X.values():
#         common = common.intersection(df.index)
#     if common.empty:
#         raise ValueError("Aucun échantillon commun entre omiques et labels.")
#     y = y.loc[common]
#     X = {k: df.loc[common] for k, df in X.items()}

#     groups=pd.Series([e.split("_")[3] for e in y.index], index=y.index)

#     return X, None, y, None, groups, "binary"

# X_train, _, y_train, _, groups, task_type = load_maigane("./data/Maigane data")



print("Shapes après chargement du dataset OOL :")
print({k: df.shape for k, df in X_train.items()})
print("Labels :", y_train.shape, y_train.unique())
# =========================================================================
# 5. Lancement du benchmark
# =========================================================================
#save_path = "/Users/noeamar/Documents/Stanford/Benchmarks results/Test Bootstrap + Prop artificial/OOL KO fdr_thresh_0.01/50 boot + 1 prop"
save_path = "/Users/noeamar/Documents/Stanford/Benchmarks results/OOL/Test with EGA/OOL FDR_001 KO_25_folds"
if os.path.exists(save_path):
    shutil.rmtree(save_path, ignore_errors=True)

print("Run CV on Covid dataset")
print("IDs :", ids)

multi_omic_stabl_cv_noe(
    data_dict=X_train,
    y=y_train,
    outer_splitter=outer_cv,
    estimators=estimators,
    task_type=task_type,
    save_path=save_path,
    model_chosen="xgboost",
    importance_method="gain",
    outer_groups=ids,
    early_fusion=False,
    late_fusion=True,
    n_iter_lf=1000,
    models=models
)

In [36]:
# ===============================
# 1. Imports et configuration générale
# ===============================
import os, re
import shutil
import numpy as np
import pandas as pd
from stabl import data
from stabl.multi_omic_pipelines import multi_omic_stabl_cv, multi_omic_stabl_cv_noe, multi_omic_stabl_noe
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from stabl.stabl import Stabl
from stabl.adaptive import ALogitLasso, ALasso
from sklearn.linear_model import Lasso
from sklearn.base import clone
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupShuffleSplit

np.random.seed(42)

# ===============================
# 2. Définition des splits de validation croisée
# ===============================
# Outer CV pour l'évaluation globale, inner CV pour la recherche d'hyperparamètres
outer_cv = GroupShuffleSplit(n_splits=100, test_size=0.2, random_state=42)
#outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=20, random_state=42)
inner_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

# ===============================
# 3. Définition des estimateurs et grilles d'hyperparamètres
# ===============================
artificial_type = "knockoff"  # ou "random_permutation" ou "knockoff"

# Classification

# Lasso
lasso = LogisticRegression(penalty="l1", class_weight="balanced", max_iter=int(1e6), solver="liblinear", random_state=42)
lasso_cv = GridSearchCV(lasso, param_grid={"C": np.logspace(-2, 2, 30)}, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

# ALasso
alasso = ALogitLasso(n_iter_lasso=2, penalty="l1", solver="liblinear", class_weight="balanced", max_iter=int(1e6), random_state=42)
alasso_cv = GridSearchCV(alasso, param_grid={"C": np.logspace(-3, 2, 20)}, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

# ElasticNet
en = LogisticRegression(penalty='elasticnet', solver='saga', class_weight='balanced', max_iter=int(1e4), random_state=42)
en_params = {"C": np.logspace(-2, 1, 10), "l1_ratio": [0.5, 0.7, 0.9]}
en_cv = GridSearchCV(en, param_grid=en_params, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

# RandomForest
rf = RandomForestClassifier(random_state=42, max_features=0.2)
rf_grid = {"max_depth": [3, 5, 7, 9, 11]}
rf_cv = GridSearchCV(rf, scoring='roc_auc', param_grid=rf_grid, cv=inner_cv, n_jobs=-1)

# XGBoost
xgb = XGBClassifier(n_estimators=200, learning_rate=0.01, subsample=0.8, colsample_bytree=0.8, colsample_bynode=0.9, importance_type="shap", n_jobs=-1)
xgb_grid = {"max_depth": [3, 6, 9], "reg_alpha": [0, 0.1, 0.5, 1]}
xgb_cv = GridSearchCV(xgb, scoring='roc_auc', param_grid=xgb_grid, cv=inner_cv, n_jobs=-1)

#CatBoost
# cb = CatBoostClassifier(random_state=42, iterations=500, verbose=False)
# cb_grid = {"depth": [3, 5, 7], "learning_rate": [0.01, 0.1, 0.2], "l2_leaf_reg": [1, 3, 5]}
# cb_cv = GridSearchCV(cb, scoring='roc_auc', param_grid=cb_grid, cv=inner_cv, n_jobs=-1)

# Regression 
# lasso = Lasso(max_iter=int(1e6), random_state=42)
# lasso_cv = GridSearchCV(lasso, param_grid={"alpha": np.logspace(-2, 2, 30)}, scoring="r2", cv=inner_cv, n_jobs=-1)

# # ALasso
# alasso = ALasso(max_iter=int(1e6), random_state=42)
# alasso_cv = GridSearchCV(alasso, scoring="r2", param_grid={"alpha": np.logspace(-2, 2, 30)}, cv=inner_cv, n_jobs=-1)

# xgb = XGBRegressor(n_estimators=200, importance_type="gain", n_jobs=-1)
# xgb_grid = {"max_depth": [3, 6, 9], "reg_alpha": [0, 0.1, 0.5, 1]}
# xgb_cv = GridSearchCV(xgb, scoring='roc_auc', param_grid=xgb_grid, cv=inner_cv, n_jobs=-1)


# ===============================
# 4. Définition des estimateurs STABL
# ===============================

# STABL Binaires

stabl_lasso = Stabl(
    base_estimator=lasso,
    n_bootstraps=1000,
    artificial_type=artificial_type,
    artificial_proportion=1,
    replace=False,
    fdr_threshold_range=np.arange(0.1, 1, 0.01),
    sample_fraction=0.5,
    random_state=42,
    lambda_grid={"C": np.linspace(0.01, 1, 10)},
    verbose=1
)

stabl_alasso = Stabl(
    base_estimator=alasso,
    n_bootstraps=1000,
    artificial_type=artificial_type,
    artificial_proportion=1.0,
    replace=False,
    fdr_threshold_range=np.arange(0.1, 1, 0.01),
    sample_fraction=0.5,
    random_state=42,
    lambda_grid={"C": np.logspace(-2, 2, 10)},
    verbose=1,
)

stabl_en = clone(stabl_lasso).set_params(
    base_estimator=en,
    n_bootstraps=500,
    lambda_grid=[{"C": np.logspace(-2, 1, 5), "l1_ratio": [0.5, 0.9]}],
    verbose=1
)

stabl_rf = clone(stabl_lasso).set_params(
    base_estimator=rf,
    n_bootstraps=200,
    lambda_grid=rf_grid,
    verbose=1
)

stabl_xgb = clone(stabl_lasso).set_params(
    base_estimator=xgb,
    #bootstrap_threshold="median",
    n_bootstraps=250,
    fdr_threshold_range=np.arange(0.1, 1, 0.01),
    lambda_grid=[xgb_grid],
    verbose=1
)

# # STABL Régression

# stabl_lasso = Stabl(
#     base_estimator=lasso,
#     n_bootstraps=500,
#     artificial_type=artificial_type,
#     artificial_proportion=1.,
#     replace=False,
#     fdr_threshold_range=np.arange(0.01, 1, 0.01),
#     sample_fraction=0.5,
#     random_state=42,
#     lambda_grid={"alpha": np.logspace(0, 2, 10)},
#     verbose=1
# )

# stabl_alasso = clone(stabl_lasso).set_params(
#     base_estimator=alasso,
#     lambda_grid={"alpha": np.logspace(0, 2, 10)},
#     verbose=1
# )

# stabl_xgb = clone(stabl_lasso).set_params(
#     base_estimator=xgb,
#     n_bootstraps=150,
#     lambda_grid=[xgb_grid],
#     verbose=1
# )

# stabl_cb = clone(stabl_lasso).set_params(
#     base_estimator=cb,
#     n_bootstraps=100,
#     lambda_grid=[cb_grid],
#     verbose=1
# )

# ===============================
# 5. Dictionnaire des estimateurs pour le benchmark
# ===============================
estimators = {
    "lasso": lasso_cv,
    "alasso": alasso_cv,
    "en": en_cv,
    "xgb": xgb_cv,
    "stabl_lasso": stabl_lasso,
    "stabl_alasso": stabl_alasso,
    "stabl_en": stabl_en,
    "stabl_xgb": stabl_xgb,
}

models = [
    "STABL Lasso",
    "STABL ALasso",
    "STABL XGBoost",
#    "STABL ElasticNet",
]


# ===============================
# 6. Chargement des données
# ===============================
#X_train, X_valid, y_train, y_valid, ids, task_type = data.load_covid_19("/Users/noeamar/Documents/Stanford/data/COVID-19")
#X_train, X_valid, y_train, y_valid, ids, task_type = data.load_ssi("data/Biobank SSI")
#X_train, X_valid, y_train, y_valid, ids, task_type = data.load_cfrna("data/CFRNA")
#X_train, X_valid, y_train, y_valid, ids, task_type = data.load_dream("data/Dream")
#X_train, X_valid, y_train, y_valid, ids, task_type = data.load_onset_of_labor_EGA("data/Onset of Labor")

# # Covid-19
# # Assainir les labels
# # Harmoniser X_train / X_valid pour avoir mêmes lignes + colonnes
# if isinstance(X_train, dict):
#     common_keys = sorted(set(X_train) & set(X_valid))
#     X_train = {k: X_train[k].reindex(index=y_train.index, columns=X_train[k].columns.intersection(X_valid[k].columns)) for k in common_keys}
#     X_valid = {k: X_valid[k].reindex(index=y_valid.index, columns=X_train[k].columns) for k in common_keys}
# else:
#     common_cols = X_train.columns.intersection(X_valid.columns)
#     X_train = X_train.reindex(index=y_train.index, columns=common_cols)
#     X_valid = X_valid.reindex(index=y_valid.index, columns=common_cols)

# y = y_train.astype(int)
# vals = sorted(set(y.unique()))
# print("Labels uniques:", vals)
# assert set(vals) <= {0, 1}

# print("Shapes avant préprocessing :")
# print({k: df.shape for k, df in X_train.items()})  # aperçu des shapes

# # Séparer CyTOF et Proteomics
# cyto = X_train["CyTOF"]
# prot = X_train["Proteomics"].copy()

# # Application du préprocessing sur chaque omique, en gardant tous les patients
# # y inchangé (même index que data.load_ssi)
# y_train = y
# common = y_train.index.copy()
# for df in (cyto, prot):
#     common = common.intersection(df.index)
# X_train = {"CyTOF" : cyto.loc[common], "Proteomics" : prot.loc[common]}

# print("Shapes après préprocessing :")
# print({k: df.shape for k, df in X_train.items()})
# print("Labels :", y_train.shape, y_train.unique())

# # 2. Isoler le gros DataFrame CyTOF + DataFrame Proteomics
# cytof_full = X_train.pop("CyTOF")          # retire temporairement
# prot_df    = X_train.pop("Proteomics")     # retire temporairement

# # 3. Décomposer CyTOF en 3 sous-ensembles
# stim_map = {"unstim": "Unstim", "lps": "LPS", "il246": "IL246", "tnf": "TNF"}
# X_train  = {}                              # on reconstruit le dict final

# for short, nice in stim_map.items():
#     regex = re.compile(fr"(^|_)({short})(_|$)", flags=re.IGNORECASE)
#     cols  = [c for c in cytof_full.columns if regex.search(c)]
#     if not cols:
#         raise ValueError(f"Aucune colonne pour la stimulation « {short} »")
#     X_train[nice] = cytof_full[cols].copy()

# # 4. Ajouter Proteomics comme 4ᵉ entrée
# X_train["Proteomics"] = prot_df.copy()

# 5. Garder UNIQUEMENT les échantillons communs aux 4 DataFrames + y
# common = y_train.index.copy()
# for df in X_train.values():
#     common = common.intersection(df.index)
# if common.empty:
#     raise ValueError("Aucun échantillon commun entre omiques et labels.")

# for k in X_train:
#     X_train[k] = X_train[k].loc[common].copy()
# y_train = y_train.loc[common]

# print({k: df.shape for k, df in X_train.items()})  # aperçu des shapes

# Maigane dataset

from os.path import join
from pathlib import Path

def load_maigane(data_path, label_mode):
    """
    Charge le dataset MAIGANE et renvoie:
      X_dict, None, y, None, groups, "binary"
    """
    from os.path import join

    def _sanitize_cols(cols):
        def clean(c):
            s = str(c).strip().strip('"').strip("'")
            s = re.sub(r'[\\/:*?"<>|]+', '_', s)
            s = re.sub(r'\s+', ' ', s).strip().replace(' ', '_')
            s = re.sub(r'_+', '_', s)
            if s == '' or s.lower().startswith('unnamed'):
                s = 'feature'
            return s
        cleaned = [clean(c) for c in cols]
        seen, out = {}, []
        for s in cleaned:
            if s in seen:
                out.append(f"{s}__{seen[s]}")
                seen[s] += 1
            else:
                seen[s] = 1
                out.append(s)
        return out

    def _read_omic(path):
        df0 = pd.read_csv(path, engine="python", sep=None,
                          na_values=["NA", "NaN", ""], keep_default_na=True)
        first = df0.columns[0]
        cond_unnamed = first in ("", " ", "Unnamed: 0", "Unnamed: 0.1", "Unnamed: 1")
        cond_slide = df0[first].astype(str).str.startswith("slide").any()

        if cond_unnamed or cond_slide:
            df = df0.set_index(first)
        else:
            df = pd.read_csv(path, engine="python", sep=None,
                             na_values=["NA", "NaN", ""], keep_default_na=True, index_col=0)

        idx = pd.Index(df.index).astype(str)
        idx = idx.str.strip().str.strip('"').str.strip("'")
        df.index = idx
        df = df[~df.index.duplicated(keep="first")]
        df.columns = _sanitize_cols(df.columns)
        return df.select_dtypes(include=[np.number])

    # Labels
    y1 = pd.read_csv(Path(data_path, "ControlVsEM.csv"), index_col=0)["EM"]
    if y1.dtype.kind not in "iu":
        y1 = y1.map({"Control": 0, "EM": 1}).astype("Int64")
    else:
        y1 = y1.astype("Int64")

    y2 = pd.read_csv(Path(data_path, "StageI&IIVsStageIII&IV.csv"), index_col=0)["Stage"]
    if y2.dtype.kind not in "iu":
        y2 = y2.map({"I&II": 0, "III&IV": 1}).astype("Int64")
    else:
        y2 = y2.astype("Int64")

    y1.index = pd.Index(y1.index).astype(str).str.strip().str.strip('"').str.strip("'")
    y2.index = pd.Index(y2.index).astype(str).str.strip().str.strip('"').str.strip("'")
    if label_mode == "control_vs_em":
        y = y1.dropna().astype(int)
    elif label_mode == "control_vs_mild":
        idx = sorted(set(y1.index[y1 == 0]).union(set(y2.index[y2 == 0])))
        y = pd.Series(pd.NA, index=idx, dtype="Int64")
        y.loc[y1.index[y1 == 0]] = 0
        y.loc[y2.index[y2 == 0]] = 1
    elif label_mode == "control_vs_severe":
        idx = sorted(set(y1.index[y1 == 0]).union(set(y2.index[y2 == 1])))
        y = pd.Series(pd.NA, index=idx, dtype="Int64")
        y.loc[y1.index[y1 == 0]] = 0
        y.loc[y2.index[y2 == 1]] = 1
    elif label_mode == "mild_vs_severe":
        idx = sorted(y2.dropna().index)
        y = y2.loc[idx]
    else:
        raise ValueError(f"label_mode inconnu: {label_mode}")

    y = y.dropna().astype(int)

    # Omics
    file_map = {
        #"CellDensities": "EMIMCmdiop_celldensities.csv",
        #"Functional":    "EMIMCmdiop_functional.csv",
        #"Metavariables": "EMIMCmdiop_metavariables.csv",
        "Neighborhood":  "EMIMCmdiop_neighborhood.csv",
    }
    X = {k: _read_omic(Path(data_path, fname)) for k, fname in file_map.items()}

    # Align
    common = pd.Index(y.index)
    for df in X.values():
        common = common.intersection(df.index)
    if common.empty:
        raise ValueError("Aucun échantillon commun entre les omiques et y.")
    y = y.loc[common]
    X = {k: df.loc[common] for k, df in X.items()}

    # Groups
    def _extract_group(e: str):
        parts = re.split(r'[_\s]+', e)
        return parts[3] if len(parts) > 3 else parts[-1]
    groups = pd.Series({_id: _extract_group(_id) for _id in y.index})

    return X, None, y, None, groups, "binary"

X_train, _, y_train, _, groups, task_type = load_maigane("./data/Maigane data", "mild_vs_severe")


print("Shapes après chargement du dataset  :")
print({k: df.shape for k, df in X_train.items()})
print("Labels :", y_train.shape, y_train.unique())
# =========================================================================
# 5. Lancement du benchmark
# =========================================================================
#save_path = "/Users/noeamar/Documents/Stanford/Benchmarks results/Test Bootstrap + Prop artificial/OOL KO fdr_thresh_0.01/50 boot + 1 prop"
#save_path = "/Users/noeamar/Documents/Stanford/Benchmarks results/Maigane data/Control vs Mild/Final feature selection Functional + Metavariables"
save_path = "/Users/noeamar/Documents/Stanford/Benchmarks results/Maigane data/Mild vs Severe/Final feature selection Neighborhood KO"
if os.path.exists(save_path):
    shutil.rmtree(save_path, ignore_errors=True)

print("Run CV on Covid dataset")
print("IDs :", groups)

multi_omic_stabl_noe(
    data_dict=X_train,
    y=y_train,
    estimators=estimators,
    task_type=task_type,
    save_path=save_path,
    model_chosen="random_forest",
    importance_method="shap",
    groups=groups,
    early_fusion=False,
    models=models,
    n_iter_lf=1000,
)

Shapes après chargement du dataset  :
{'Neighborhood': (109, 2747)}
Labels : (109,) [1 0]
Run CV on Covid dataset
IDs : slide10_P01endoIMC_mgd_UC724_1_4    UC724
slide10_P01endoIMC_mgd_UC724_2_5    UC724
slide10_P01endoIMC_mgd_UC724_3_6    UC724
slide11_P01endoIMC_mgd_CNC12_2_5    CNC12
slide11_P01endoIMC_mgd_CNC12_3_6    CNC12
                                    ...  
slide8_P01endoIMC_mgd_CNC29_2_5     CNC29
slide8_P01endoIMC_mgd_CNC29_3_6     CNC29
slide9_P01endoIMC_mgd_UC657_1_4     UC657
slide9_P01endoIMC_mgd_UC657_2_5     UC657
slide9_P01endoIMC_mgd_UC657_3_6     UC657
Length: 109, dtype: object
Stabl progress: 100%|██████████| 12/12 [02:27<00:00, 12.26s/it]


0it [00:00, ?it/s]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Selected features for STABL Lasso: 7
Selected features for STABL ALasso: 2
Selected features for STABL XGBoost: 0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



{}

In [ ]:
from pathlib import Path
import os
import re
import numpy as np
import pandas as pd
import shutil
from stabl import data
from stabl.multi_omic_pipelines import multi_omic_stabl_cv, multi_omic_stabl_cv_noe, multi_omic_stabl_noe
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from stabl.stabl import Stabl
from stabl.adaptive import ALogitLasso, ALasso
from sklearn.linear_model import Lasso
from sklearn.base import clone
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupShuffleSplit

# ====== PARAMS À AJUSTER ======
data_path       = "/Users/noeamar/Documents/Stanford/data/Maigane data"
label_mode      = "mild_vs_severe"                # "control_vs_mild" | "control_vs_severe" | "mild_vs_severe"
artificial_type = "knockoff"                        # "random_permutation" | "knockoff"
model_chosen    = "xgboost"                          # utilisé pour le modèle final dans STABL
importance_method = "shap"                           # pour STABL XGBoost (et variantes arbres si tu en ajoutes)

# Dossier de sortie unique par run (timestamp → pas d’écrasement, écriture progressive)
root_results = Path("/Users/noeamar/Documents/Stanford/Benchmarks results/Maigane data")
run_name     = f"{label_mode}_{model_chosen}_{artificial_type}_{importance_method}_early_fusion"
results_dir  = root_results / run_name
(results_dir / "Training CV").mkdir(parents=True, exist_ok=True)
(results_dir / "Summary").mkdir(parents=True, exist_ok=True)

print(">> RESULTS DIR:", results_dir)

def load_maigane(data_path, label_mode):
    """
    Charge le dataset MAIGANE et renvoie:
      X_dict, None, y, None, groups, "binary"
    """
    from os.path import join

    def _sanitize_cols(cols):
        def clean(c):
            s = str(c).strip().strip('"').strip("'")
            s = re.sub(r'[\\/:*?"<>|]+', '_', s)
            s = re.sub(r'\s+', ' ', s).strip().replace(' ', '_')
            s = re.sub(r'_+', '_', s)
            if s == '' or s.lower().startswith('unnamed'):
                s = 'feature'
            return s
        cleaned = [clean(c) for c in cols]
        seen, out = {}, []
        for s in cleaned:
            if s in seen:
                out.append(f"{s}__{seen[s]}")
                seen[s] += 1
            else:
                seen[s] = 1
                out.append(s)
        return out

    def _read_omic(path):
        df0 = pd.read_csv(path, engine="python", sep=None,
                          na_values=["NA", "NaN", ""], keep_default_na=True)
        first = df0.columns[0]
        cond_unnamed = first in ("", " ", "Unnamed: 0", "Unnamed: 0.1", "Unnamed: 1")
        cond_slide = df0[first].astype(str).str.startswith("slide").any()

        if cond_unnamed or cond_slide:
            df = df0.set_index(first)
        else:
            df = pd.read_csv(path, engine="python", sep=None,
                             na_values=["NA", "NaN", ""], keep_default_na=True, index_col=0)

        idx = pd.Index(df.index).astype(str)
        idx = idx.str.strip().str.strip('"').str.strip("'")
        df.index = idx
        df = df[~df.index.duplicated(keep="first")]
        df.columns = _sanitize_cols(df.columns)
        return df.select_dtypes(include=[np.number])

    # Labels
    y1 = pd.read_csv(Path(data_path, "ControlVsEM.csv"), index_col=0)["EM"]
    if y1.dtype.kind not in "iu":
        y1 = y1.map({"Control": 0, "EM": 1}).astype("Int64")
    else:
        y1 = y1.astype("Int64")

    y2 = pd.read_csv(Path(data_path, "StageI&IIVsStageIII&IV.csv"), index_col=0)["Stage"]
    if y2.dtype.kind not in "iu":
        y2 = y2.map({"I&II": 0, "III&IV": 1}).astype("Int64")
    else:
        y2 = y2.astype("Int64")

    y1.index = pd.Index(y1.index).astype(str).str.strip().str.strip('"').str.strip("'")
    y2.index = pd.Index(y2.index).astype(str).str.strip().str.strip('"').str.strip("'")
    if label_mode == "control_vs_em":
        y = y1.dropna().astype(int)
    elif label_mode == "control_vs_mild":
        idx = sorted(set(y1.index[y1 == 0]).union(set(y2.index[y2 == 0])))
        y = pd.Series(pd.NA, index=idx, dtype="Int64")
        y.loc[y1.index[y1 == 0]] = 0
        y.loc[y2.index[y2 == 0]] = 1
    elif label_mode == "control_vs_severe":
        idx = sorted(set(y1.index[y1 == 0]).union(set(y2.index[y2 == 1])))
        y = pd.Series(pd.NA, index=idx, dtype="Int64")
        y.loc[y1.index[y1 == 0]] = 0
        y.loc[y2.index[y2 == 1]] = 1
    elif label_mode == "mild_vs_severe":
        idx = sorted(y2.dropna().index)
        y = y2.loc[idx]
    else:
        raise ValueError(f"label_mode inconnu: {label_mode}")

    y = y.dropna().astype(int)

    # Omics
    file_map = {
        "CellDensities": "EMIMCmdiop_celldensities.csv",
        "Functional":    "EMIMCmdiop_functional.csv",
        "Metavariables": "EMIMCmdiop_metavariables.csv",
        "Neighborhood":  "EMIMCmdiop_neighborhood.csv",
    }
    X = {k: _read_omic(Path(data_path, fname)) for k, fname in file_map.items()}

    # Align
    common = pd.Index(y.index)
    for df in X.values():
        common = common.intersection(df.index)
    if common.empty:
        raise ValueError("Aucun échantillon commun entre les omiques et y.")
    y = y.loc[common]
    X = {k: df.loc[common] for k, df in X.items()}

    # Groups
    def _extract_group(e: str):
        parts = re.split(r'[_\s]+', e)
        return parts[3] if len(parts) > 3 else parts[-1]
    groups = pd.Series({_id: _extract_group(_id) for _id in y.index})

    return X, None, y, None, groups, "binary"

print(">> Loader prêt")

print(">> Chargement des données…")
X_train, X_val, y_train, y_val, groups, task_type = load_maigane(data_path, label_mode)

print(">> Shapes par omique :")
for k, df in X_train.items():
    print(f"  - {k:14s} : {df.shape}")
print(">> y_train distribution :", np.bincount(y_train.values))
print(">> #groups :", len(pd.Series(groups).unique()))

# CV externe group-aware : 25 folds
outer_cv = GroupShuffleSplit(n_splits=100, test_size=0.2, random_state=42)
print(">> Outer CV: GroupShuffleSplit(n_splits=25, test_size=0.2)")

# CV interne pour les baselines
inner_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

print(">> Construction des estimateurs et STABL…")

# Lasso
lasso = LogisticRegression(penalty="l1", class_weight="balanced",
                           max_iter=int(1e6), solver="liblinear", random_state=42)
lasso_cv = GridSearchCV(lasso, param_grid={"C": np.logspace(-2, 2, 30)},
                        scoring="roc_auc", cv=inner_cv, n_jobs=-1)

# ALogitLasso tags (compat sklearn)
def _alogit_class_tags(cls, estimator=None):
    return LogisticRegression().get_tags()
def _alogit_get_tags(self):
    return LogisticRegression().get_tags()
ALogitLasso.__sklearn_tags__ = _alogit_class_tags
ALogitLasso._get_tags        = _alogit_get_tags

alasso = ALogitLasso(
    n_iter_lasso=2,
    penalty="l1",
    solver="liblinear",
    class_weight="balanced",
    max_iter=int(1e6),
    random_state=42,
)
alasso_cv = GridSearchCV(alasso, param_grid={"C": np.logspace(-3, 2, 20)},
                         scoring="roc_auc", cv=inner_cv, n_jobs=-1)

# ElasticNet logit
en = LogisticRegression(penalty='elasticnet', solver='saga', class_weight='balanced',
                        max_iter=int(1e4), random_state=42)
en_params = {"C": np.logspace(-2, 1, 10), "l1_ratio": [0.5, 0.7, 0.9]}
en_cv = GridSearchCV(en, param_grid=en_params, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

# XGBoost tags (compat sklearn)
@classmethod
def _xgb_cls_tags(cls, estimator=None):
    return LogisticRegression().get_tags()
def _xgb_cls_get_tags(self):
    return LogisticRegression().get_tags()
XGBClassifier.__sklearn_tags__ = _xgb_cls_tags
XGBClassifier._get_tags        = _xgb_cls_get_tags

xgb = XGBClassifier(
    n_estimators=200, learning_rate=0.01,
    subsample=0.8, colsample_bytree=0.8, colsample_bynode=0.9,
    importance_type=importance_method, n_jobs=-1
)
xgb_grid = {"max_depth": [3, 6, 9], "reg_alpha": [0, 0.5, 1, 2, 5]}
xgb_cv = GridSearchCV(xgb, scoring='roc_auc', param_grid=xgb_grid, cv=inner_cv, n_jobs=-1)

# STABL (bootstraps)
stabl_lasso = Stabl(
    base_estimator=lasso,
    n_bootstraps=500,
    artificial_type=artificial_type,
    artificial_proportion=1.0,
    replace=False,
    fdr_threshold_range=np.arange(0.01, 1, 0.01),
    sample_fraction=0.5,
    random_state=42,
    lambda_grid={"C": np.linspace(0.01, 1, 10)},
    verbose=1
)
stabl_alasso = Stabl(
    base_estimator=alasso,
    n_bootstraps=500,
    artificial_type=artificial_type,
    artificial_proportion=1.0,
    replace=False,
    fdr_threshold_range=np.arange(0.01, 1, 0.01),
    sample_fraction=0.5,
    random_state=42,
    lambda_grid={"C": np.logspace(-2, 2, 10)},
    verbose=1,
)
stabl_en = clone(stabl_lasso).set_params(
    base_estimator=en,
    n_bootstraps=250,
    lambda_grid={"C": np.logspace(-2, 1, 5), "l1_ratio": [0.5, 0.9]},
    verbose=1
)
stabl_xgb = clone(stabl_lasso).set_params(
    base_estimator=xgb,
    n_bootstraps=100,
    lambda_grid=xgb_grid,
    verbose=1
)

estimators = {
    "lasso": lasso_cv,
    "alasso": alasso_cv,
    "en": en_cv,
    "xgb": xgb_cv,
    "stabl_lasso": stabl_lasso,
    "stabl_alasso": stabl_alasso,
    "stabl_en": stabl_en,
    "stabl_xgb": stabl_xgb,
}

models = [
#    "STABL Lasso",
#    "STABL ALasso",
#    "STABL ElasticNet",
#    "STABL XGBoost",
    "lasso",
    "alasso",
    "xgb",
]

print(">> Estimators prêts.")

print("\n==================== LANCEMENT STABL CV (25 folds) ====================")
print(f">> results_dir: {results_dir}")
print(f">> models: {models}")
print(f">> artificial_type: {artificial_type} | importance_method: {importance_method}")
print("======================================================================\n")

predictions_dict = multi_omic_stabl_cv_noe(
    data_dict=X_train,
    y=y_train,
    outer_splitter=outer_cv,         # 25 splits
    estimators=estimators,
    task_type="binary",
    model_chosen=model_chosen,
    save_path=str(results_dir),
    importance_method=importance_method,
    models=models,
    outer_groups=pd.Series(groups),  # respect des groupes
    early_fusion=True,              # multi-omic → pas d'early fusion
    late_fusion=False,                # on fait de la late fusion
    n_iter_lf=100000
)

print("\n==================== FIN DU RUN ====================")
print("Les résultats/plots ont été écrits progressivement dans :", results_dir)



>> RESULTS DIR: /Users/noeamar/Documents/Stanford/Benchmarks results/Maigane data/control_vs_mild_xgboost_knockoff_shap_early_fusion
>> Loader prêt
>> Chargement des données…
>> Shapes par omique :
  - CellDensities  : (143, 172)
  - Functional     : (143, 1344)
  - Metavariables  : (143, 239)
  - Neighborhood   : (143, 2747)
>> y_train distribution : [111  32]
>> #groups : 50
>> Outer CV: GroupShuffleSplit(n_splits=25, test_size=0.2)
>> Construction des estimateurs et STABL…
>> Estimators prêts.

==================== LANCEMENT STABL CV (25 folds) ====================
>> results_dir: /Users/noeamar/Documents/Stanford/Benchmarks results/Maigane data/control_vs_mild_xgboost_knockoff_shap_early_fusion
>> models: ['lasso', 'alasso', 'xgb']
>> artificial_type: knockoff | importance_method: shap

114 train samples, 29 test samples:   0%|          | 0/100 [00:00<?, ?it/s]Fitting of Early Fusion models
~~~~~~~~~~~~~~~~~~~
This fold: 0 features selected for lasso
This fold: 0 features selected 

ValueError: Input contains NaN.

In [ ]:
import time
import json
import traceback
from pathlib import Path
import numpy as np
import pandas as pd
import re
from stabl import data
from stabl.multi_omic_pipelines import multi_omic_stabl_cv_noe
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from stabl.stabl import Stabl
from stabl.adaptive import ALogitLasso, ALasso
from xgboost import XGBClassifier
from sklearn.model_selection import GroupShuffleSplit
import os
import sys
import shutil
import logging 

from datetime import datetime

def log(msg, *a):
    ts = datetime.now().strftime("%H:%M:%S")
    print(f"[{ts}] {msg}", *a, flush=True)

def sprint_header(title):
    bar = "=" * 80
    log(bar)
    log(title)
    log(bar)



# ============================================================================
# CONFIGS — SINGLE-OMIC + mild_vs_severe + KO/RP × SHAP/total_cover
# ============================================================================
DATA_PATH    = "/Users/noeamar/Documents/Stanford/data/Maigane data"
BASE_RESULTS = Path("/Users/noeamar/Documents/Stanford/Benchmarks results/Maigane data/Single-omic combos")

LABEL_MODE   = "control_vs_mild"                 # demandé
ARTIFICIALS  = ["knockoff", "random_permutation"]
IMP_METHODS  = ["shap", "total_cover"]          # demandé
MODEL_CHOSEN = "random_forest"

# 25 folds group-aware
outer_cv = GroupShuffleSplit(n_splits=30, test_size=0.2, random_state=42)
inner_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

# ============================================================================
# HELPERS
# ============================================================================

def make_estimators(artificial_type: str, importance: str, inner_cv):
    """
    Construit baselines + STABL pour CE combo.
    - Baseline XGB: on NE met pas importance_type='shap' (non supporté nativement par XGBoost).
      -> si importance == 'shap', on laisse l'importance native (p.ex. 'total_cover') côté baseline.
    - STABL-XGB: l'argument 'importance_method' est passé via multi_omic_stabl_cv_noe,
      qui injecte la bonne méthode (SHAP/total_cover) dans les STABL arbre/boosting.
    """
    sprint_header(f"CONSTRUCTION DES ESTIMATEURS | artificial={artificial_type} | importance={importance}")

    # Baselines (GridSearch)
    log("-> Préparation GridSearch Lasso…")
    lasso = LogisticRegression(penalty="l1", class_weight="balanced",
                               max_iter=int(1e6), solver="liblinear", random_state=42)
    lasso_grid = {"C": np.logspace(-2, 2, 30)}
    lasso_cv = GridSearchCV(lasso, param_grid=lasso_grid, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

    # ALogitLasso shim sklearn tags
    def _alogit_class_tags(cls, estimator=None):
        return LogisticRegression().get_tags()
    def _alogit_get_tags(self):
        return LogisticRegression().get_tags()
    ALogitLasso.__sklearn_tags__ = _alogit_class_tags
    ALogitLasso._get_tags        = _alogit_get_tags

    log("-> Préparation GridSearch A-Logit-Lasso…")
    alasso = ALogitLasso(
        n_iter_lasso=2, penalty="l1", solver="liblinear",
        class_weight="balanced", max_iter=int(1e6), random_state=42,
    )
    alasso_grid = {"C": np.logspace(-3, 2, 20)}
    alasso_cv = GridSearchCV(alasso, param_grid=alasso_grid, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

    log("-> Préparation GridSearch ElasticNet (LogReg ‘saga’)…")
    en = LogisticRegression(penalty='elasticnet', solver='saga', class_weight='balanced',
                            max_iter=int(1e4), random_state=42)
    en_grid = {"C": np.logspace(-2, 1, 10), "l1_ratio": [0.5, 0.7, 0.9]}
    en_cv = GridSearchCV(en, param_grid=en_grid, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

    # XGBoost (baseline)
    @classmethod
    def _xgb_cls_tags(cls, estimator=None):
        return LogisticRegression().get_tags()
    def _xgb_cls_get_tags(self):
        return LogisticRegression().get_tags()
    XGBClassifier.__sklearn_tags__ = _xgb_cls_tags
    XGBClassifier._get_tags        = _xgb_cls_get_tags

    log("-> Préparation GridSearch XGBoost…")
    xgb = XGBClassifier(
        n_estimators=200, learning_rate=0.01,
        subsample=0.8, colsample_bytree=0.8, colsample_bynode=0.9,
        n_jobs=-1
    )
    # importance_type: seulement si supporté nativement par XGB
    if importance in {"weight", "gain", "cover", "total_gain", "total_cover"}:
        xgb.set_params(importance_type=importance)
    xgb_grid = {"max_depth": [3, 6, 9], "reg_alpha": [0, 0.5, 1, 2, 5]}
    xgb_cv = GridSearchCV(xgb, scoring='roc_auc', param_grid=xgb_grid, cv=inner_cv, n_jobs=-1)

    # STABL
    log("-> Préparation STABL Lasso…")
    # STABL (bootstraps)
    stabl_lasso = Stabl(
        base_estimator=lasso,
        n_bootstraps=500,
        artificial_type=artificial_type,
        artificial_proportion=1.0,
        replace=False,
        fdr_threshold_range=np.arange(0.01, 1, 0.01),
        sample_fraction=0.5,
        random_state=42,
        lambda_grid={"C": np.linspace(0.01, 1, 10)},
        verbose=1
    )
    stabl_alasso = Stabl(
        base_estimator=alasso,
        n_bootstraps=500,
        artificial_type=artificial_type,
        artificial_proportion=1.0,
        replace=False,
        fdr_threshold_range=np.arange(0.01, 1, 0.01),
        sample_fraction=0.5,
        random_state=42,
        lambda_grid={"C": np.logspace(-2, 2, 10)},
        verbose=1,
    )
    stabl_en = clone(stabl_lasso).set_params(
        base_estimator=en,
        n_bootstraps=250,
        lambda_grid={"C": np.logspace(-2, 1, 5), "l1_ratio": [0.5, 0.9]},
        verbose=1
    )
    stabl_xgb = clone(stabl_lasso).set_params(
        base_estimator=xgb,
        n_bootstraps=100,
        lambda_grid=xgb_grid,
        verbose=1
    )
    estimators = {
        "lasso": lasso_cv,
        "alasso": alasso_cv,
        "en": en_cv,
        "xgb": xgb_cv,
        "stabl_lasso": stabl_lasso,
        "stabl_alasso": stabl_alasso,
        "stabl_en": stabl_en,
        "stabl_xgb": stabl_xgb,
    }
    log("✔ Estimators construits.")
    return estimators


def run_one_single_omic(label_mode: str, artificial_type: str, importance: str, omic_name: str,
                        run_idx: int, total_runs: int):
    sprint_header(
        f"RUN {run_idx}/{total_runs} | label_mode={label_mode} | omic={omic_name} "
        f"| artificial={artificial_type} | importance={importance}"
    )

    # charge X/y/groups
    log("Chargement des données…")
    X_full, _, y_train, _, groups, _ = load_maigane(DATA_PATH, label_mode)
    if omic_name not in X_full:
        raise ValueError(f"Omique '{omic_name}' introuvable dans X (disponibles: {list(X_full.keys())})")

    # mono-omique
    X_sub = {omic_name: X_full[omic_name]}

    log(f"   {omic_name} shape: {tuple(X_sub[omic_name].shape)}")
    log(f"   y size: {len(y_train)} | y distribution (0/1): {np.bincount(y_train.values)}")
    uniq_groups = pd.Series(groups).astype(str)
    log(f"   #groups: {uniq_groups.nunique()} | exemples: {uniq_groups.unique()[:8]}")

    # Estimators pour CE combo
    estimators = make_estimators(artificial_type, importance, inner_cv)

    # Modèles à lancer :
    # - Toujours STABL XGBoost (la méthode d'importance réelle est passée via importance_method)
    # - Baseline XGB UNIQUEMENT si importance est native XGB (pas 'shap')
    models = ["STABL Lasso", "STABL ALasso", "STABL XGBoost"]

    # dossier de résultat spécifique
    results_dir = BASE_RESULTS / f"{label_mode}_{omic_name}_{MODEL_CHOSEN}_{artificial_type}_{importance}_single_omic"
    (results_dir / "Training CV").mkdir(parents=True, exist_ok=True)
    (results_dir / "Summary").mkdir(parents=True, exist_ok=True)
    log(f"Dossier résultats: '{results_dir}'")

    # journaux
    meta = {
        "label_mode": label_mode,
        "artificial_type": artificial_type,
        "importance": importance,
        "model_chosen": MODEL_CHOSEN,
        "n_splits": outer_cv.n_splits,
        "test_size": 0.2,
        "timestamp_start": time.strftime("%Y-%m-%d %H:%M:%S"),
        "omics_in_combo": [omic_name],
    }
    with open(results_dir / "RUN_STARTED.json", "w") as f:
        json.dump(meta, f, indent=2)
    log("RUN_STARTED.json écrit.")

    # CV
    log("Configuration CV externe (GroupShuffleSplit):")
    log(f"   n_splits={outer_cv.n_splits}, test_size=0.2, random_state=42")
    log("Configuration CV interne (RepeatedStratifiedKFold):")
    log("   n_splits=5, n_repeats=5, random_state=42")

    # Lancement
    log("DÉMARRAGE multi_omic_stabl_cv_noe… (mono-omique, EF/LF désactivées)")
    t0 = time.time()
    _ = multi_omic_stabl_cv_noe(
        data_dict=X_sub,
        y=y_train,
        outer_splitter=outer_cv,
        estimators=estimators,
        task_type="binary",
        model_chosen=MODEL_CHOSEN,
        save_path=str(results_dir),
        importance_method=importance,   # 'shap' ou 'total_cover' (injection côté STABL)
        models=models,
        outer_groups=pd.Series(groups),
        early_fusion=False,             # single-omic → EF inutile
        late_fusion=True,              # single-omic → LF inutile
        n_iter_lf=100000,               # ignoré si LF=False
    )
    dt = time.time() - t0
    log(f"multi_omic_stabl_cv_noe TERMINÉ en {dt/60:.1f} min")

    # post-run
    meta["timestamp_end"] = time.strftime("%Y-%m-%d %H:%M:%S")
    with open(results_dir / "RUN_DONE.json", "w") as f:
        json.dump(meta, f, indent=2)
    log("RUN_DONE.json écrit.")

    scores_cv = results_dir / "Training CV" / "Scores training CV.csv"
    if scores_cv.exists():
        try:
            df_scores = pd.read_csv(scores_cv)
            log(f"Scores CV trouvés ({scores_cv.name}) — top lignes:")
            log(df_scores.head(3).to_string(index=False))
        except Exception as e:
            log(f"Impossible de lire '{scores_cv.name}' (pas bloquant): {e}")
    else:
        log(f"ATTENTION: '{scores_cv.name}' introuvable pour ce combo (peut être normal si interrompu).")

    log(f"✓ FIN → {results_dir}")


# ============================================================================
# ORCHESTRATION — enchaîne toutes les omiques × (KO,RP) × (SHAP,total_cover)
# ============================================================================
sprint_header("CHARGEMENT DES OMICS POUR LISTER LES NOMS")
X_all, _, y_tmp, _, groups_tmp, _ = load_maigane(DATA_PATH, LABEL_MODE)
omics_list = list(X_all.keys())
log(f"Omics détectées: {omics_list}")
del X_all, y_tmp, groups_tmp

all_runs = []
for omic in omics_list:
    for art in ARTIFICIALS:
        for imp in IMP_METHODS:
            all_runs.append((omic, art, imp))

sprint_header(f"TOTAL RUNS À EXÉCUTER: {len(all_runs)} (25 folds chacun)")

global_start = time.time()
for idx, (omic_name, art, imp) in enumerate(all_runs, start=1):
    try:
        run_one_single_omic(LABEL_MODE, art, imp, omic_name, idx, len(all_runs))
    except Exception as e:
        out = BASE_RESULTS / f"{LABEL_MODE}_{omic_name}_{MODEL_CHOSEN}_{art}_{imp}_single_omic"
        out.mkdir(parents=True, exist_ok=True)
        with open(out / "RUN_FAILED.txt", "w") as f:
            f.write(str(e) + "\n\n" + traceback.format_exc())
        log(f"✗ ECHEC sur {omic_name} | {art} | {imp} (voir RUN_FAILED.txt)")

elapsed = time.time() - global_start
sprint_header(f"TOUS LES RUNS TERMINÉS en {elapsed/60:.1f} min")


[14:00:22] ================================================================================
[14:00:22] CHARGEMENT DES OMICS POUR LISTER LES NOMS
[14:00:22] ================================================================================
[14:00:23] Omics détectées: ['CellDensities', 'Functional', 'Metavariables', 'Neighborhood']
[14:00:23] ================================================================================
[14:00:23] TOTAL RUNS À EXÉCUTER: 16 (25 folds chacun)
[14:00:23] ================================================================================
[14:00:23] ================================================================================
[14:00:23] RUN 1/16 | label_mode=mild_vs_severe | omic=CellDensities | artificial=knockoff | importance=shap
[14:00:23] ================================================================================
[14:00:23] Chargement des données…
[14:00:24]    CellDensities shape: (109, 172)
[14:00:24]    y size: 109 | y distribution (0/1): [32 77]

In [22]:
import time
import json
import traceback
from pathlib import Path
import numpy as np
import pandas as pd
import re
from stabl import data
from stabl.multi_omic_pipelines import multi_omic_stabl_cv_noe
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from stabl.stabl import Stabl
from stabl.adaptive import ALogitLasso, ALasso
from xgboost import XGBClassifier
from sklearn.base import clone
from sklearn.model_selection import GroupShuffleSplit
import os
import sys
import shutil
import logging
from datetime import datetime

# ====================== LOG / HEADER ======================
def log(msg, *a):
    ts = datetime.now().strftime("%H:%M:%S")
    print(f"[{ts}] {msg}", *a, flush=True)

def sprint_header(title):
    bar = "=" * 80
    log(bar); log(title); log(bar)

# ============================================================================
# CONFIGS — LATE-FUSION MULTI-OMIC + mild_vs_severe + KO/RP × SHAP/total_cover
# ============================================================================
DATA_PATH    = "/Users/noeamar/Documents/Stanford/data/Maigane data"
BASE_RESULTS = Path("/Users/noeamar/Documents/Stanford/Benchmarks results/Maigane data/Control vs Mild/Late-fusion multi-omic combos")

LABEL_MODE   = "control_vs_mild"                  # idem
ARTIFICIALS  = ["knockoff", "random_permutation"]
IMP_METHODS  = ["shap", "total_cover"]
MODEL_CHOSEN = "random_forest"                   # tag pour tes répertoires

# CV externe/interne
outer_cv = GroupShuffleSplit(n_splits=30, test_size=0.2, random_state=42)
inner_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

# ============================================================================
# HELPERS (identiques, mais réutilisés pour le multi-omic)
# ============================================================================

def load_maigane(data_path, label_mode):
    """
    Charge le dataset MAIGANE et renvoie:
      X_dict, None, y, None, groups, "binary"
    """
    from os.path import join

    def _sanitize_cols(cols):
        def clean(c):
            s = str(c).strip().strip('"').strip("'")
            s = re.sub(r'[\\/:*?"<>|]+', '_', s)
            s = re.sub(r'\s+', ' ', s).strip().replace(' ', '_')
            s = re.sub(r'_+', '_', s)
            if s == '' or s.lower().startswith('unnamed'):
                s = 'feature'
            return s
        cleaned = [clean(c) for c in cols]
        seen, out = {}, []
        for s in cleaned:
            if s in seen:
                out.append(f"{s}__{seen[s]}")
                seen[s] += 1
            else:
                seen[s] = 1
                out.append(s)
        return out

    def _read_omic(path):
        df0 = pd.read_csv(path, engine="python", sep=None,
                          na_values=["NA", "NaN", ""], keep_default_na=True)
        first = df0.columns[0]
        cond_unnamed = first in ("", " ", "Unnamed: 0", "Unnamed: 0.1", "Unnamed: 1")
        cond_slide = df0[first].astype(str).str.startswith("slide").any()

        if cond_unnamed or cond_slide:
            df = df0.set_index(first)
        else:
            df = pd.read_csv(path, engine="python", sep=None,
                             na_values=["NA", "NaN", ""], keep_default_na=True, index_col=0)

        idx = pd.Index(df.index).astype(str)
        idx = idx.str.strip().str.strip('"').str.strip("'")
        df.index = idx
        df = df[~df.index.duplicated(keep="first")]
        df.columns = _sanitize_cols(df.columns)
        return df.select_dtypes(include=[np.number])

    # Labels
    y1 = pd.read_csv(Path(data_path, "ControlVsEM.csv"), index_col=0)["EM"]
    if y1.dtype.kind not in "iu":
        y1 = y1.map({"Control": 0, "EM": 1}).astype("Int64")
    else:
        y1 = y1.astype("Int64")

    y2 = pd.read_csv(Path(data_path, "StageI&IIVsStageIII&IV.csv"), index_col=0)["Stage"]
    if y2.dtype.kind not in "iu":
        y2 = y2.map({"I&II": 0, "III&IV": 1}).astype("Int64")
    else:
        y2 = y2.astype("Int64")

    y1.index = pd.Index(y1.index).astype(str).str.strip().str.strip('"').str.strip("'")
    y2.index = pd.Index(y2.index).astype(str).str.strip().str.strip('"').str.strip("'")
    if label_mode == "control_vs_em":
        y = y1.dropna().astype(int)
    elif label_mode == "control_vs_mild":
        idx = sorted(set(y1.index[y1 == 0]).union(set(y2.index[y2 == 0])))
        y = pd.Series(pd.NA, index=idx, dtype="Int64")
        y.loc[y1.index[y1 == 0]] = 0
        y.loc[y2.index[y2 == 0]] = 1
    elif label_mode == "control_vs_severe":
        idx = sorted(set(y1.index[y1 == 0]).union(set(y2.index[y2 == 1])))
        y = pd.Series(pd.NA, index=idx, dtype="Int64")
        y.loc[y1.index[y1 == 0]] = 0
        y.loc[y2.index[y2 == 1]] = 1
    elif label_mode == "mild_vs_severe":
        idx = sorted(y2.dropna().index)
        y = y2.loc[idx]
    else:
        raise ValueError(f"label_mode inconnu: {label_mode}")

    y = y.dropna().astype(int)

    # Omics
    file_map = {
        "CellDensities": "EMIMCmdiop_celldensities.csv",
        "Functional":    "EMIMCmdiop_functional.csv",
        "Metavariables": "EMIMCmdiop_metavariables.csv",
        "Neighborhood":  "EMIMCmdiop_neighborhood.csv",
    }
    X = {k: _read_omic(Path(data_path, fname)) for k, fname in file_map.items()}

    # Align
    common = pd.Index(y.index)
    for df in X.values():
        common = common.intersection(df.index)
    if common.empty:
        raise ValueError("Aucun échantillon commun entre les omiques et y.")
    y = y.loc[common]
    X = {k: df.loc[common] for k, df in X.items()}

    # Groups
    def _extract_group(e: str):
        parts = re.split(r'[_\s]+', e)
        return parts[3] if len(parts) > 3 else parts[-1]
    groups = pd.Series({_id: _extract_group(_id) for _id in y.index})

    return X, None, y, None, groups, "binary"

def make_estimators(artificial_type: str, importance: str, inner_cv):
    """
    Construit baselines + STABL pour CE combo.
    """
    sprint_header(f"CONSTRUCTION DES ESTIMATEURS | artificial={artificial_type} | importance={importance}")

    # ----- Baselines (GridSearch) -----
    log("-> Préparation GridSearch Lasso…")
    lasso = LogisticRegression(penalty="l1", class_weight="balanced",
                               max_iter=int(1e6), solver="liblinear", random_state=42)
    lasso_grid = {"C": np.logspace(-2, 2, 30)}
    lasso_cv = GridSearchCV(lasso, param_grid=lasso_grid, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

    # ALogitLasso: shim sklearn tags
    def _alogit_class_tags(cls, estimator=None):
        return LogisticRegression().get_tags()
    def _alogit_get_tags(self):
        return LogisticRegression().get_tags()
    ALogitLasso.__sklearn_tags__ = _alogit_class_tags
    ALogitLasso._get_tags        = _alogit_get_tags

    log("-> Préparation GridSearch A-Logit-Lasso…")
    alasso = ALogitLasso(
        n_iter_lasso=2, penalty="l1", solver="liblinear",
        class_weight="balanced", max_iter=int(1e6), random_state=42,
    )
    alasso_grid = {"C": np.logspace(-3, 2, 20)}
    alasso_cv = GridSearchCV(alasso, param_grid=alasso_grid, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

    log("-> Préparation GridSearch ElasticNet (LogReg ‘saga’)…")
    en = LogisticRegression(penalty='elasticnet', solver='saga', class_weight='balanced',
                            max_iter=int(1e4), random_state=42)
    en_grid = {"C": np.logspace(-2, 1, 10), "l1_ratio": [0.5, 0.7, 0.9]}
    en_cv = GridSearchCV(en, param_grid=en_grid, scoring="roc_auc", cv=inner_cv, n_jobs=-1)

    # XGBoost baseline (+ shim tags)
    @classmethod
    def _xgb_cls_tags(cls, estimator=None):
        return LogisticRegression().get_tags()
    def _xgb_cls_get_tags(self):
        return LogisticRegression().get_tags()
    XGBClassifier.__sklearn_tags__ = _xgb_cls_tags
    XGBClassifier._get_tags        = _xgb_cls_get_tags

    log("-> Préparation GridSearch XGBoost…")
    xgb = XGBClassifier(
        n_estimators=200, learning_rate=0.01,
        subsample=0.8, colsample_bytree=0.8, colsample_bynode=0.9,
        n_jobs=-1
    )
    # importance_type uniquement si support natif XGB
    if importance in {"weight", "gain", "cover", "total_gain", "total_cover"}:
        xgb.set_params(importance_type=importance)
    xgb_grid = {"max_depth": [3, 6, 9], "reg_alpha": [0, 0.5, 1, 2, 5]}
    xgb_cv = GridSearchCV(xgb, scoring='roc_auc', param_grid=xgb_grid, cv=inner_cv, n_jobs=-1)

    # ----- STABL bootstrappé -----
    log("-> Préparation STABL (Lasso / ALasso / EN / XGB)…")
    stabl_lasso = Stabl(
        base_estimator=lasso,
        n_bootstraps=500,
        artificial_type=artificial_type,
        artificial_proportion=1.0,
        replace=False,
        fdr_threshold_range=np.arange(0.01, 1, 0.01),
        sample_fraction=0.5,
        random_state=42,
        lambda_grid={"C": np.linspace(0.01, 1, 10)},
        verbose=1
    )
    stabl_alasso = Stabl(
        base_estimator=alasso,
        n_bootstraps=500,
        artificial_type=artificial_type,
        artificial_proportion=1.0,
        replace=False,
        fdr_threshold_range=np.arange(0.01, 1, 0.01),
        sample_fraction=0.5,
        random_state=42,
        lambda_grid={"C": np.logspace(-2, 2, 10)},
        verbose=1,
    )
    stabl_en = clone(stabl_lasso).set_params(
        base_estimator=en,
        n_bootstraps=250,
        lambda_grid={"C": np.logspace(-2, 1, 5), "l1_ratio": [0.5, 0.9]},
        verbose=1
    )
    stabl_xgb = clone(stabl_lasso).set_params(
        base_estimator=xgb,
        n_bootstraps=100,
        lambda_grid=xgb_grid,
        verbose=1
    )

    estimators = {
        "lasso": lasso_cv,
        "alasso": alasso_cv,
        "en": en_cv,
        "xgb": xgb_cv,
        "stabl_lasso": stabl_lasso,
        "stabl_alasso": stabl_alasso,
        "stabl_en": stabl_en,
        "stabl_xgb": stabl_xgb,
    }
    log("✔ Estimators construits.")
    return estimators

# ============================================================================
# RUNNER — Late-fusion multi-omic (toutes omiques)
# ============================================================================
def run_multi_omic_lf(label_mode: str, artificial_type: str, importance: str,
                      run_idx: int, total_runs: int):
    sprint_header(
        f"RUN {run_idx}/{total_runs} | label_mode={label_mode} "
        f"| artificial={artificial_type} | importance={importance} | MODE=Late-fusion multi-omic"
    )

    # ----- Chargement des données multi-omic -----
    log("Chargement des données (toutes omiques)…")
    X_full, _, y_train, _, groups, _ = load_maigane(DATA_PATH, label_mode)
    assert isinstance(X_full, dict) and len(X_full) > 0, "X_full doit être un dict omique -> DataFrame"
    log(f"Omics détectées: {list(X_full.keys())}")

    # Quelques stats rapides
    any_df = next(iter(X_full.values()))
    log(f"   Exemple omique shape: {tuple(any_df.shape)}")
    log(f"   y size: {len(y_train)} | y distribution (0/1): {np.bincount(y_train.values)}")
    uniq_groups = pd.Series(groups).astype(str)
    log(f"   #groups: {uniq_groups.nunique()} | exemples: {uniq_groups.unique()[:8]}")

    # ----- Estimators pour CE combo -----
    estimators = make_estimators(artificial_type, importance, inner_cv)

    # Modèles à lancer (mêmes STABL que ton script single-omic ; LF activée dans l'appel)
    models = ["STABL Lasso", "STABL ALasso", "STABL XGBoost"]

    # ----- Dossier résultats -----
    results_dir = BASE_RESULTS / f"{label_mode}_ALL_OMICS_{MODEL_CHOSEN}_{artificial_type}_{importance}_late_fusion_multi_omic"
    (results_dir / "Training CV").mkdir(parents=True, exist_ok=True)
    (results_dir / "Summary").mkdir(parents=True, exist_ok=True)
    log(f"Dossier résultats: '{results_dir}'")

    # ----- Meta run -----
    meta = {
        "label_mode": label_mode,
        "artificial_type": artificial_type,
        "importance": importance,
        "model_chosen": MODEL_CHOSEN,
        "n_splits": outer_cv.n_splits,
        "test_size": 0.2,
        "timestamp_start": time.strftime("%Y-%m-%d %H:%M:%S"),
        "omics_in_combo": list(X_full.keys()),
        "fusion": "late_fusion_multi_omic",
    }
    with open(results_dir / "RUN_STARTED.json", "w") as f:
        json.dump(meta, f, indent=2)
    log("RUN_STARTED.json écrit.")

    # ----- Logs CV -----
    log("Configuration CV externe (GroupShuffleSplit):")
    log(f"   n_splits={outer_cv.n_splits}, test_size=0.2, random_state=42")
    log("Configuration CV interne (RepeatedStratifiedKFold):")
    log("   n_splits=5, n_repeats=5, random_state=42")

    # ----- Lancement Late-Fusion -----
    log("DÉMARRAGE multi_omic_stabl_cv_noe… (multi-omic, late_fusion=True)")
    t0 = time.time()
    _ = multi_omic_stabl_cv_noe(
        data_dict=X_full,                 # <— toutes les omiques
        y=y_train,
        outer_splitter=outer_cv,
        estimators=estimators,
        task_type="binary",
        model_chosen=MODEL_CHOSEN,
        save_path=str(results_dir),
        importance_method=importance,     # 'shap' ou 'total_cover' (injection côté STABL)
        models=models,
        outer_groups=pd.Series(groups),
        early_fusion=False,               # EF désactivée
        late_fusion=True,                 # LF activée
        n_iter_lf=100000,                 # itérations/échantillons de LF si utilisé par ta fonction
    )
    dt = time.time() - t0
    log(f"multi_omic_stabl_cv_noe TERMINÉ en {dt/60:.1f} min")

    # ----- Post-run -----
    meta["timestamp_end"] = time.strftime("%Y-%m-%d %H:%M:%S")
    with open(results_dir / "RUN_DONE.json", "w") as f:
        json.dump(meta, f, indent=2)
    log("RUN_DONE.json écrit.")

    scores_cv = results_dir / "Training CV" / "Scores training CV.csv"
    if scores_cv.exists():
        try:
            df_scores = pd.read_csv(scores_cv)
            log(f"Scores CV trouvés ({scores_cv.name}) — top lignes:")
            log(df_scores.head(3).to_string(index=False))
        except Exception as e:
            log(f"Impossible de lire '{scores_cv.name}' (pas bloquant): {e}")
    else:
        log(f"ATTENTION: '{scores_cv.name}' introuvable pour ce combo (peut être normal si interrompu).")

    log(f"✓ FIN → {results_dir}")

# ============================================================================
# ORCHESTRATION — enchaîne (KO, RP) × (SHAP, total_cover) en LF multi-omic
# ============================================================================
sprint_header("PRÉPARATION DES RUNS LATE-FUSION MULTI-OMIC")
# Ping unique pour lister les omiques (et fail fast si souci)
try:
    X_all, _, y_tmp, _, groups_tmp, _ = load_maigane(DATA_PATH, LABEL_MODE)
    log(f"Omics détectées: {list(X_all.keys())}")
    del X_all, y_tmp, groups_tmp
except Exception as e:
    log(f"ERREUR load_maigane en préparation: {e}")
    raise

all_runs = [(art, imp) for art in ARTIFICIALS for imp in IMP_METHODS]
sprint_header(f"TOTAL RUNS À EXÉCUTER: {len(all_runs)} (30 folds group-aware chacun)")

global_start = time.time()
for idx, (art, imp) in enumerate(all_runs, start=1):
    try:
        run_multi_omic_lf(LABEL_MODE, art, imp, idx, len(all_runs))
    except Exception as e:
        out = BASE_RESULTS / f"{LABEL_MODE}_ALL_OMICS_{MODEL_CHOSEN}_{art}_{imp}_late_fusion_multi_omic"
        out.mkdir(parents=True, exist_ok=True)
        with open(out / "RUN_FAILED.txt", "w") as f:
            f.write(str(e) + "\n\n" + traceback.format_exc())
        log(f"✗ ECHEC sur combo | {art} | {imp} (voir RUN_FAILED.txt)")

elapsed = time.time() - global_start
sprint_header(f"TOUS LES RUNS TERMINÉS en {elapsed/60:.1f} min")


[11:32:11] ================================================================================
[11:32:11] PRÉPARATION DES RUNS LATE-FUSION MULTI-OMIC
[11:32:11] ================================================================================
[11:32:12] Omics détectées: ['CellDensities', 'Functional', 'Metavariables', 'Neighborhood']
[11:32:12] ================================================================================
[11:32:12] TOTAL RUNS À EXÉCUTER: 4 (30 folds group-aware chacun)
[11:32:12] ================================================================================
[11:32:12] ================================================================================
[11:32:12] RUN 1/4 | label_mode=control_vs_mild | artificial=knockoff | importance=shap | MODE=Late-fusion multi-omic
[11:32:12] ================================================================================
[11:32:12] Chargement des données (toutes omiques)…
[11:32:12] Omics détectées: ['CellDensities', 'Functional', 'Met

0it [00:00, ?it/s]

~~~~~~~~~~~~~~~~~~~
This fold: 119 features selected for STABL Lasso
This fold: 114 features selected for STABL ALasso
This fold: 11 features selected for STABL XGBoost
~~~~~~~~~~~~~~~~~~~

113 train samples, 30 test samples:   3%|▎         | 1/30 [06:29<3:08:01, 389.02s/it]

Fitting of STABL Lasso
Stabl progress: 100%|██████████| 10/10 [00:02<00:00,  3.45it/s]
STABL Lasso finished on CellDensities (109 samples); 2 features selected
Fitting of STABL ALasso
Stabl progress: 100%|██████████| 10/10 [00:04<00:00,  2.32it/s]
STABL ALasso finished on CellDensities (109 samples); 2 features selected
Fitting of STABL XGBoost
Stabl progress: 100%|██████████| 15/15 [00:38<00:00,  2.60s/it]
STABL XGBoost finished on CellDensities (109 samples); 6 features selected
Fitting of STABL Lasso
Stabl progress: 100%|██████████| 10/10 [00:06<00:00,  1.63it/s]
STABL Lasso finished on Functional (113 samples); 31 features selected
Fitting of STABL ALasso
Stabl progress: 100%|██████████| 10/10 [00:09<00:00,  1.05it/s]
STABL ALasso finished on Functional (113 samples); 35 features selected
Fitting of STABL XGBoost
Stabl progress: 100%|██████████| 15/15 [02:14<00:00,  8.96s/it]
STABL XGBoost finished on Functional (113 samples); 4 features selected
Fitting of STABL Lasso
Stabl progre

0it [00:00, ?it/s]

...

~~~~~~~~~~~~~~~~~~~
This fold: 5 features selected for STABL Lasso
This fold: 10 features selected for STABL ALasso
This fold: 36 features selected for STABL XGBoost
~~~~~~~~~~~~~~~~~~~

113 train samples, 30 test samples:  97%|█████████▋| 29/30 [2:31:22<05:10, 310.83s/it]

Fitting of STABL Lasso
Stabl progress: 100%|██████████| 10/10 [00:02<00:00,  3.66it/s]
STABL Lasso finished on CellDensities (106 samples); 0 features selected
Fitting of STABL ALasso
Stabl progress: 100%|██████████| 10/10 [00:04<00:00,  2.44it/s]
STABL ALasso finished on CellDensities (106 samples); 0 features selected
Fitting of STABL XGBoost
Stabl progress: 100%|██████████| 15/15 [00:28<00:00,  1.89s/it]
STABL XGBoost finished on CellDensities (106 samples); 3 features selected
Fitting of STABL Lasso
Stabl progress: 100%|██████████| 10/10 [00:06<00:00,  1.55it/s]
STABL Lasso finished on Functional (113 samples); 1 features selected
Fitting of STABL ALasso
Stabl progress: 100%|██████████| 10/10 [00:09<00:00,  1.00it/s]
STABL ALasso finished on Functional (113 samples); 1 features selected
Fitting of STABL XGBoost
Stabl progress: 100%|██████████| 15/15 [02:05<00:00,  8.37s/it]
STABL XGBoost finished on Functional (113 samples); 6 features selected
Fitting of STABL Lasso
Stabl progress

0it [00:00, ?it/s]

~~~~~~~~~~~~~~~~~~~
This fold: 7 features selected for STABL Lasso
This fold: 7 features selected for STABL ALasso
This fold: 31 features selected for STABL XGBoost
~~~~~~~~~~~~~~~~~~~

113 train samples, 30 test samples: 100%|██████████| 30/30 [2:36:27<00:00, 312.91s/it]
Saving results...


STABL Lasso slide10_P01endoIMC_mgd_UC738_1_1    0.989109
slide10_P01endoIMC_mgd_UC738_2_2    0.601801
slide10_P01endoIMC_mgd_UC738_3_3    0.004512
slide11_P01endoIMC_mgd_UC740_1_1    0.232199
slide11_P01endoIMC_mgd_UC740_2_2    0.243416
                                      ...   
slide9_P01endoIMC_mgd_NCB024_2_2    0.740326
slide9_P01endoIMC_mgd_NCB024_3_3    0.334199
slide9_P01endoIMC_mgd_UC657_1_4     0.138813
slide9_P01endoIMC_mgd_UC657_2_5     0.084498
slide9_P01endoIMC_mgd_UC657_3_6     0.000956
Length: 143, dtype: float64
STABL ALasso slide10_P01endoIMC_mgd_UC738_1_1    0.995396
slide10_P01endoIMC_mgd_UC738_2_2    0.841264
slide10_P01endoIMC_mgd_UC738_3_3    0.208253
slide11_P01endoIMC_mgd_UC740_1_1    0.232199
slide11_P01endoIMC_mgd_UC740_2_2    0.243416
                                      ...   
slide9_P01endoIMC_mgd_NCB024_2_2    0.768076
slide9_P01endoIMC_mgd_NCB024_3_3    0.221783
slide9_P01endoIMC_mgd_UC657_1_4     0.123387
slide9_P01endoIMC_mgd_UC657_2_5     0.606954
sl

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import re
from ast import literal_eval
from pathlib import Path
from typing import Dict, List, Optional, Set

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ===================== PARAMS =====================
RUN_DIRS = [
    "/Users/noeamar/Documents/Stanford/OOL Test Bootstraps et Artifical features/results_stabl_OOL_xgboost_knockoff_gain_B50_A1",
    "/Users/noeamar/Documents/Stanford/OOL Test Bootstraps et Artifical features/results_stabl_OOL_xgboost_knockoff_gain_B100_A1",
    "/Users/noeamar/Documents/Stanford/OOL Test Bootstraps et Artifical features/results_stabl_OOL_xgboost_knockoff_gain_B200_A0.5",
    "/Users/noeamar/Documents/Stanford/OOL Test Bootstraps et Artifical features/results_stabl_OOL_xgboost_knockoff_gain_B200_A1",
    "/Users/noeamar/Documents/Stanford/OOL Test Bootstraps et Artifical features/results_stabl_OOL_xgboost_knockoff_gain_B200_A2",
    "/Users/noeamar/Documents/Stanford/OOL Test Bootstraps et Artifical features/results_stabl_OOL_xgboost_knockoff_gain_B200_A5",
    "/Users/noeamar/Documents/Stanford/OOL Test Bootstraps et Artifical features/results_stabl_OOL_xgboost_knockoff_gain_B500_A1",
]

# Output folder
OUT_DIR = Path("/Users/noeamar/Documents/Stanford/OOL Test Bootstraps et Artifical features/_ool_comparisons")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Expected CSV filename (searched recursively)
CSV_NAME = "Selected Features STABL XGBoost.csv"

# Extract B and A from folder names
RE_B = re.compile(r"B(\d+)", re.IGNORECASE)
RE_A = re.compile(r"A([0-9]*\.?[0-9]+)", re.IGNORECASE)

# ===================== HELPERS =====================
def parse_list_cell(cell) -> List[str]:
    """Convert 'Fold selected features' string cell to a Python list."""
    if pd.isna(cell):
        return []
    s = str(cell).strip()
    if not s:
        return []
    if s.startswith('[') and s.endswith(']'):
        # best effort literal eval
        try:
            val = literal_eval(s)
            if isinstance(val, list):
                return [str(x) for x in val]
        except Exception:
            pass
        # fallback split
        inner = s[1:-1].strip()
        if not inner:
            return []
        return [x.strip().strip("'\"") for x in inner.split(',') if x.strip()]
    return []

def detect_selected_col(df: pd.DataFrame) -> Optional[str]:
    """Try to find the 'fold selected features' column."""
    for c in df.columns:
        cl = c.lower()
        if ("selected" in cl) and ("feat" in cl):
            return c
    return None

def get_fold_id_from_row(df: pd.DataFrame, i: int) -> int:
    """Try to infer a fold id from index/columns; fallback to row number."""
    idx_val = df.index[i]
    if isinstance(idx_val, str):
        m = re.search(r'(\d+)', idx_val)
        if m:
            return int(m.group(1))
    if isinstance(idx_val, (int, np.integer)):
        return int(idx_val)
    for c in df.columns:
        if "fold" in c.lower():
            val = df.iloc[i][c]
            if isinstance(val, str):
                m = re.search(r'(\d+)', val)
                if m:
                    return int(m.group(1))
            elif isinstance(val, (int, np.integer, float, np.floating)):
                try:
                    return int(val)
                except Exception:
                    pass
    return int(i)

def is_artificial_feature(name: str) -> bool:
    """
    Heuristic to detect artificial/knockoff features.
    Adjust the patterns if your naming differs.
    """
    s = name.lower()
    return (
        s.startswith("a_") or
        s.startswith("artif") or
        "artificial" in s or
        "knock" in s or
        s.endswith("_ko") or
        ".ko" in s
    )

def try_read_csv_with_index(path: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(path, index_col=0)
    except Exception:
        return pd.read_csv(path)

def jaccard(a: Set[str], b: Set[str]) -> float:
    if not a and not b:
        return 1.0
    return len(a & b) / max(1, len(a | b))

def collect_run_info(run_dir: Path) -> Optional[Dict]:
    """
    For a run folder:
      - read all 'Selected Features STABL XGBoost.csv' files recursively
      - aggregate features per fold (union if multiple CSVs)
      - compute union across folds, artificial proportion,
        and mean intra-run Jaccard (off-diagonal mean)
    """
    csvs = sorted(run_dir.glob(f"**/{CSV_NAME}"))
    if not csvs:
        return None

    # Parse B / A from folder name
    mB = RE_B.search(run_dir.name)
    mA = RE_A.search(run_dir.name)
    B = int(mB.group(1)) if mB else None
    A = float(mA.group(1)) if mA else None

    fold_sets: Dict[int, Set[str]] = {}

    for f in csvs:
        df = try_read_csv_with_index(f)
        sel_col = detect_selected_col(df)
        if sel_col is None:
            continue

        for i in range(len(df)):
            feats = set(parse_list_cell(df.iloc[i][sel_col]))
            fold_id = get_fold_id_from_row(df, i)
            fold_sets[fold_id] = fold_sets.get(fold_id, set()) | feats

    if not fold_sets:
        return None

    # Union across folds
    union_set = set().union(*fold_sets.values())
    n_union = len(union_set)

    # Artificial proportion (on union)
    if n_union > 0:
        n_art = sum(1 for x in union_set if is_artificial_feature(x))
        art_prop = n_art / n_union
    else:
        art_prop = np.nan

    # Intra-run Jaccard (mean off-diagonal)
    keys = sorted(fold_sets)
    k = len(keys)
    if k >= 2:
        J = np.zeros((k, k), float)
        for i, fi in enumerate(keys):
            Si = fold_sets[fi]
            for j, fj in enumerate(keys):
                Sj = fold_sets[fj]
                J[i, j] = jaccard(Si, Sj)
        mask = ~np.eye(k, dtype=bool)
        mean_intra_jaccard = float(J[mask].mean())
    else:
        mean_intra_jaccard = np.nan

    # Per-fold feature counts (useful for sanity checks)
    per_fold_sizes = [len(fold_sets[fi]) for fi in keys]
    avg_features_per_fold = float(np.mean(per_fold_sizes)) if per_fold_sizes else np.nan

    return {
        "run_dir": str(run_dir),
        "name": run_dir.name,
        "B": B,
        "A": A,
        "n_folds": len(fold_sets),
        "union_n_features": n_union,
        "artificial_prop": art_prop,
        "mean_intra_jaccard": mean_intra_jaccard,
        "avg_features_per_fold": avg_features_per_fold,
        "fold_sets": fold_sets,
        "union_set": union_set,
    }

# ===================== MAIN =====================
def main():
    results = []
    for p in RUN_DIRS:
        rd = Path(p)
        info = collect_run_info(rd)
        if info:
            results.append(info)
        else:
            print(f"[WARN] No compatible CSV found in: {rd}")

    if not results:
        print("No valid run found.")
        return

    # Global summary
    keep_cols = ["name","B","A","n_folds","union_n_features","artificial_prop","mean_intra_jaccard","avg_features_per_fold","run_dir"]
    summary = pd.DataFrame([{k: v for k, v in m.items() if k in keep_cols} for m in results])
    summary.sort_values(by=["A","B"], inplace=True, ignore_index=True)
    summary.to_csv(OUT_DIR / "summary_ool.csv", index=False)
    # also a compact CSV just for plotting if needed
    summary[["B","A","union_n_features","artificial_prop","mean_intra_jaccard"]].to_csv(
        OUT_DIR / "summary_for_plots.csv", index=False
    )
    print(f"[OK] summary_ool.csv → {OUT_DIR}")

    # ============ FIGURE 1 : Unique features vs B (A=1) ============
    sub_B = summary[(summary["A"] == 1)].sort_values("B")
    if not sub_B.empty:
        plt.figure()
        plt.plot(sub_B["B"], sub_B["union_n_features"], marker="o")
        plt.xlabel("Number of bootstraps (B)")
        plt.ylabel("Unique features (union across folds)")
        plt.title("Unique features vs B (A=1)")
        plt.grid(True, alpha=0.3); plt.tight_layout()
        plt.savefig(OUT_DIR / "unique_features_vs_B_A1.png", dpi=160)
        plt.close()

    # ============ FIGURE 2 : Unique features vs A (B=200) ============
    sub_A = summary[(summary["B"] == 200)].sort_values("A")
    if not sub_A.empty:
        plt.figure()
        plt.plot(sub_A["A"], sub_A["union_n_features"], marker="o")
        plt.xlabel("Proportion of artificial features (A)")
        plt.ylabel("Unique features (union across folds)")
        plt.title("Unique features vs A (B=200)")
        plt.grid(True, alpha=0.3); plt.tight_layout()
        plt.savefig(OUT_DIR / "unique_features_vs_A_B200.png", dpi=160)
        plt.close()

    # ============ FIGURE 3 : Mean intra-run Jaccard vs A (B=200 only) ============
    sub_B200 = summary[summary["B"] == 200].sort_values("A")
    if not sub_B200.empty:
        plt.figure()
        plt.plot(sub_B200["A"], sub_B200["mean_intra_jaccard"], marker="o", label="B=200")
        plt.xlabel("Proportion of artificial features (A)")
        plt.ylabel("Mean intra-run Jaccard index")
        plt.title("Mean intra-run Jaccard index vs A (B=200)")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(OUT_DIR / "mean_intra_jaccard_vs_A_B200.png", dpi=160)
        plt.close()



    print(f"\nFigures written to: {OUT_DIR}")
    print("- unique_features_vs_B_A1.png")
    print("- unique_features_vs_A_B200.png")
    print("- mean_intra_fold_jaccard_by_config.png")

if __name__ == "__main__":
    main()


In [ ]:
# ============================================
# Top-10 features (Lasso vs XGB) + Pearson/Spearman
# Pré-requis:
# - X_train: dict avec X_train["CyTOF"], X_train["Proteomics"] (DataFrames)
# - y_train: Series binaire (0/1)
# ============================================

import numpy as np
import pandas as pd
from typing import List
from scipy.stats import spearmanr

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

# ---------- 1) Aligner les données ----------
def align_data(X_train: dict, y_train: pd.Series) -> (pd.DataFrame, pd.Series):
    common_idx = y_train.index
    for k in ["CyTOF", "Proteomics"]:
        if k in X_train and isinstance(X_train[k], pd.DataFrame):
            common_idx = common_idx.intersection(X_train[k].index)
    X = pd.concat([X_train["CyTOF"].loc[common_idx],
                   X_train["Proteomics"].loc[common_idx]], axis=1)
    y = y_train.loc[common_idx].astype(int)
    return X, y

# ---------- 2) Utilitaires corrélations ----------
def safe_pearson(x: pd.Series, y: pd.Series) -> float:
    # Pearson ~ point-biserial ici (y binaire). On gère NaN en imputant médiane locale.
    xv = x.values.astype(float)
    m = np.nanmedian(xv)
    xv = np.where(np.isnan(xv), m, xv)
    # Standard Pearson
    if np.std(xv) == 0 or np.std(y.values) == 0:
        return 0.0
    return float(np.corrcoef(xv, y.values)[0, 1])

def safe_spearman(x: pd.Series, y: pd.Series) -> float:
    xv = x.values.astype(float)
    m = np.nanmedian(xv)
    xv = np.where(np.isnan(xv), m, xv)
    rho, _ = spearmanr(xv, y.values)
    if np.isnan(rho):
        return 0.0
    return float(rho)

def corr_table(X: pd.DataFrame, y: pd.Series, features: List[str],
               score_name: str, scores: pd.Series) -> pd.DataFrame:
    rows = []
    for rank, feat in enumerate(features, start=1):
        p = safe_pearson(X[feat], y)
        s = safe_spearman(X[feat], y)
        rows.append({
            "Rank": rank,
            "Feature": feat,
            score_name: float(scores.loc[feat]),
            "Pearson": p,
            "Spearman": s
        })
    return pd.DataFrame(rows)

# ---------- 3) Lasso (LogisticRegression L1) ----------
def top10_lasso(X: pd.DataFrame, y: pd.Series, C=0.1, random_state=42) -> (List[str], pd.Series):
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler",  StandardScaler()),
        ("lasso",   LogisticRegression(
            penalty="l1",
            solver="liblinear",
            C=C,
            class_weight="balanced",
            max_iter=10000,
            random_state=random_state
        ))
    ])
    pipe.fit(X, y)
    lasso = pipe.named_steps["lasso"]
    coefs = pd.Series(lasso.coef_[0], index=X.columns)
    # tri par poids absolu (on garde le signe pour l’info)
    coefs_sorted = coefs.reindex(coefs.abs().sort_values(ascending=False).index)
    top_feats = coefs_sorted.index[:10].tolist()
    return top_feats, coefs_sorted

# ---------- 4) XGBoost (classique) ----------
def top10_xgb(X: pd.DataFrame, y: pd.Series, random_state=42) -> (List[str], pd.Series):
    xgb = XGBClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.7,
        colsample_bytree=0.7,
        colsample_bynode=0.5,
        min_child_weight=1,
        reg_lambda=1.0,
        gamma=0.0,
        tree_method="hist",
        importance_type="total_gain",  # ou "gain" selon la version
        n_jobs=-1,
        random_state=random_state
    )
    xgb.fit(X, y)
    importances = pd.Series(xgb.feature_importances_, index=X.columns)
    importances_sorted = importances.sort_values(ascending=False)
    top_feats = importances_sorted.index[:10].tolist()
    return top_feats, importances_sorted

# ---------- 5) Point d’entrée ----------
def main(X_train: dict, y_train: pd.Series, save_csv_path: str = None):
    X, y = align_data(X_train, y_train)

    # Lasso
    lasso_feats, lasso_scores = top10_lasso(X, y, C=0.1, random_state=42)
    lasso_tab = corr_table(X, y, lasso_feats, score_name="Lasso_coef", scores=lasso_scores.loc[lasso_feats])

    # XGBoost
    xgb_feats, xgb_scores = top10_xgb(X, y, random_state=42)
    xgb_tab = corr_table(X, y, xgb_feats, score_name="XGB_importance", scores=xgb_scores.loc[xgb_feats])

    # Consolidation
    lasso_tab.insert(0, "Model", "Lasso")
    xgb_tab.insert(0, "Model", "XGB")
    out = pd.concat([lasso_tab, xgb_tab], axis=0, ignore_index=True)

    # Affichage
    pd.set_option("display.max_rows", 50)
    print("\n=== Top-10 Lasso (coef + corrélations) ===")
    print(lasso_tab.to_string(index=False))
    print("\n=== Top-10 XGB (importance + corrélations) ===")
    print(xgb_tab.to_string(index=False))

    # Sauvegarde optionnelle
    if save_csv_path:
        out.to_csv(save_csv_path, index=False)
        print(f"\nTableau consolidé sauvegardé: {save_csv_path}")

    return out

# ---------- Utilisation ----------
out_df = main(X_train, y_train, save_csv_path="top10_corr_lasso_xgb.csv")
